## Preamble

In [1]:
import sys
sys.path.append('..')

import importlib
import numpy as np
import os.path
import pandas as pd
import re
import seaborn as sns
import time
import traceback
import matplotlib.pyplot as plt

from collections import Counter, defaultdict
from datetime import datetime, timedelta
from enum import Enum
from functools import partial
from itertools import chain
from tqdm import tqdm
from typing import NamedTuple


import mim.extractors.skane_1718 as skane1718
import mim.extractors.skane_1718_extract_util as s1718_util

In [2]:
from datetime import datetime

def parse_iso8601_datetime(s) -> datetime:
    return datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

In [3]:
DATA_ROOT = "/mnt/air-crypt/air-crypt-raw/andersb/data/Skane_17-18/Uttag_1"

FILES = {fname: DATA_ROOT + "/" + fname for fname in [
    "MELIOR_AntalSlutenVårddagarFörSlutenVårdtillfällen_Efter_Vardkontakt_UtskrivningDatum_Till_20191231.csv",
    "MELIOR_Åtgärder30DagarEfterVårdkontakt_2017_2018.csv",
    "MELIOR_ÅtgärderVidVårdkontakt_2017_2018.csv",
    "MELIOR_Diagnoser30DagarEfterVårdkontakt_2017_2018.csv",
    "MELIOR_Diagnoser5ÅrFöreVårdkontakt_2017_2018.csv",
    "MELIOR_DiagnoserVidVårdkontakt_2017_2018.csv",
    "MELIOR_FörstaInläggningsavdelningenInom30Dagar.csv",
    "MELIOR_InneliggandeVårdtidInom30Dagar.csv",
    "MELIOR_LabanalyserInom24TimmarFrånAnkomst.csv",
    "MELIOR_OrdinationerSamtUtdelningarEttÅrFöreAnkomst.csv",
    "MELIOR_OrdinationerSamtUtdelningarInom24TimmarFrånAnkomst.csv",
    "MELIOR_PatientAtgarder_Efter_Vardkontakt_UtskrivningDatum_Till_20191231.csv",
    "MELIOR_PatientDiagnoser_Efter_Vardkontakt_UtskrivningDatum_Till_20191231.csv",
    "MELIOR_PreliminärBedömningDiagnosPåAkutenVidVårdkontakt_2017_2018.csv",
    "MELIOR_SistaUtskrivningsavdelningenInom30Dagar_2017_2018.csv",
    "MELIOR_SlutenVårdtillfällen_Efter_Vardkontakt_UtskrivningDatum_Till_20191231.csv",
    "MIKROBIOLOGI_Sammanställning_Analyser_2017.csv",
    "MIKROBIOLOGI_Sammanställning_Analyser_2018.csv",
    "MIKROBIOLOGI_Sammanställning_Analyser_2019.csv",
    "MIKROBIOLOGI_Sammanställning_Negativa_2017.csv",
    "MIKROBIOLOGI_Sammanställning_Negativa_2018.csv",
    "MIKROBIOLOGI_Sammanställning_Negativa_2019.csv",
    "MIKROBIOLOGI_Sammanställning_Positiva_2017.csv",
    "MIKROBIOLOGI_Sammanställning_Positiva_2018.csv",
    "MIKROBIOLOGI_Sammanställning_Positiva_2019.csv",
    "PASIVA_IntensivvårdInom30Dagar.csv",
    "PATIENTLIGGAREN_Vårdkontakter_2017_2018.csv",
    "PATIENTLIGGAREN_Vårdkontakter_2017_2018_Triagekategorier.csv",
    "PATIENTLIGGAREN_Vårdkontakter_Återbesök_till_191231.csv",
    "PATIENTLIGGAREN_Vårdkontakter_Januari2019.csv",
    "PATIENTLIGGAREN_Vårdkontakter_Januari2019_Triagekategorier.csv",
    "SCB_Ekelund_LEV_BoDeSO2012.csv",
    "SCB_Ekelund_LEV_BoDeSO2013.csv",
    "SCB_Ekelund_LEV_BoDeSO2014.csv",
    "SCB_Ekelund_LEV_BoDeSO2015.csv",
    "SCB_Ekelund_LEV_BoDeSO2016.csv",
    "SCB_Ekelund_LEV_BoDeSO2017.csv",
    "SCB_Ekelund_LEV_BoDeSO2018.csv",
    "SCB_Ekelund_LEV_Fodelseland.csv",
    "SCB_Ekelund_LEV_LISA2012.csv",
    "SCB_Ekelund_LEV_LISA2013.csv",
    "SCB_Ekelund_LEV_LISA2014.csv",
    "SCB_Ekelund_LEV_LISA2015.csv",
    "SCB_Ekelund_LEV_LISA2016.csv",
    "SCB_Ekelund_LEV_LISA2017.csv",
    "SCB_Ekelund_LEV_LISA2018.csv",
    "SCB_Ekelund_LEV_Nyckel.csv",
    "SECTRA_export2012_2015A.csv",
    "SECTRA_export2016_2018A.csv",
    "SOS_R_DORS_24129_2020.csv",
    "SOS_T_R_LMED_24129_2020.csv",
    "SOS_T_T_T_R_PAR_OV_24129_2020.csv",
    "SOS_T_T_T_R_PAR_SV_24129_2020.csv",
    "SVAR_Hela_Skåne_17_18.csv"
]}

## Liggaren

In [4]:
liggaren = pd.read_csv(FILES["PATIENTLIGGAREN_Vårdkontakter_2017_2018.csv"], sep="|", encoding="ISO-8859-1")
liggaren["Ambulans_Arrival_Liggare_AB"] = (liggaren["UppföljningParameter_text"] == "Ambulans").astype(int)
liggaren.head()

,KontaktId,Alias,AktuellSjukhusId,Sjukhus_Namn,BesokOrsakId,BesokOrsak_Kod,BesokOrsak_Beskrivning,Vardkontakt_InskrivningDatum,Vardkontakt_UtskrivningDatum,Vardkontakt_PatientAlderVidInskrivning,...,Utskriven till,Inläggningsavdelning,Vårddtid på akuten i minuter,Tid till läkare på akuten i minuter,Första läkarkontakten på akuten,Avvikit mot rekommendation,Inlagd,Process_text,UppföljningParameter_text,Ambulans_Arrival_Liggare_AB
0,K1054989,10237978,SE162321000255-O10532,Centralsjukhuset Kristianstad,281.0,InfLokal,"Lokal infektion RETTS-A 47, RETTS-P 141",2017-03-05 22:37:00,2017-03-06 00:29:00,56,...,Hemmet,NaN,112.0,25.0,2017-03-05 23:02:00,0,0,-,NaN,0
1,K1055321,10099865,SE162321000255-O11016,Lasarettet Trelleborg,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-03-06 11:12:00,2017-03-06 14:44:00,52,...,Hemmet,NaN,212.0,115.0,2017-03-06 13:07:00,0,0,-,NaN,0
2,K1110785,10261943,SE162321000255-O16571,Ystads lasarett,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-05-09 16:40:00,2017-05-09 21:36:00,64,...,Hemmet,NaN,296.0,240.0,2017-05-09 20:40:00,0,0,Ankomst,NaN,0
3,K1110933,10159007,SE162321000255-O16571,Ystads lasarett,300.0,SkrotSy,Underlivsbesvär hos män 17,2017-05-09 19:39:00,2017-05-10 01:17:00,19,...,Avvikit,NaN,338.0,20.0,2017-05-09 19:59:00,1,0,Ankomst,NaN,0
4,K1111161,10131344,SE162321000255-O12014,Helsingborgs lasarett,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-05-10 05:18:00,2017-05-10 05:41:00,60,...,Hemmet,Egenvård,23.0,NaN,NaN,0,0,-,NaN,0


In [5]:
liggaren_followup = pd.read_csv(FILES["PATIENTLIGGAREN_Vårdkontakter_Återbesök_till_191231.csv"], sep="|", encoding="ISO-8859-1")
liggaren_followup.drop_duplicates(["Alias", "Vardkontakt_InskrivningDatum"], inplace=True)

## Duplicates

When ignoring KontaktId, there is a small number of visits that occur more than once. Below we list their Aliases and how to drop.

(37 patients, 41 visits to drop)

In [6]:
# Show list
display("Before dropping:" + str(len(liggaren)))
display(liggaren[liggaren.drop("KontaktId", axis=1).duplicated(keep=False)].drop("KontaktId", axis=1)["Alias"].value_counts())
duplicate_wo_kontakt_id_index = [i for (i, b) in enumerate(liggaren.drop("KontaktId", axis=1).duplicated()) if b]
liggaren.drop(duplicate_wo_kontakt_id_index, inplace=True)
display("After dropping: " + str(len(liggaren)))

'Before dropping:632785'

10188455    4
10119029    3
10314582    3
10114672    2
10286650    2
10316004    2
10271012    2
10231708    2
10238151    2
10317054    2
10176080    2
10318221    2
10073405    2
10174101    2
10138116    2
10321104    2
10187874    2
10202696    2
10057608    2
10233266    2
10267412    2
10166164    2
10029124    2
10098673    2
10149844    2
10043896    2
10214049    2
10020221    2
10273081    2
10239666    2
10211420    2
10202150    2
10175208    2
10281236    2
10123680    2
10045619    2
10062902    2
Name: Alias, dtype: int64

'After dropping: 632744'

In [7]:
liggar_alias = set(liggaren["Alias"])
liggar_kontaktid = set(liggaren["KontaktId"])
#liggaren["Alias"].value_counts()

In [ ]:
list(liggaren)

In [ ]:
liggaren["Inlagd"].value_counts()

In [ ]:
for k,v in (liggaren["BesokOrsak_Kod"] + "~" + liggaren["BesokOrsak_Beskrivning"]).value_counts().items():
    print(f'{k}\t{v}')

## Preliminär bedömning

In [8]:
prelim = pd.read_csv(FILES["MELIOR_PreliminärBedömningDiagnosPåAkutenVidVårdkontakt_2017_2018.csv"], sep="|", encoding="ISO-8859-1")
prelim.head()

,KontaktId,Alias,AktivitetTyp,Diagnostyp,VårdtillfälleFörDiagnos_VardformText,PatientDiagnos_Kod,PatientDiagnos_Beskrivning,PatientDiagnos_ModifieradDatum
0,K1000057,10032954,Akutkliniken Läk,Huvuddiagnos,Slutenvård,R559,Svimning och kollaps,2017-01-03 09:43:36
1,K1055021,10083200,Akutkliniken Läk,Huvuddiagnos,Slutenvård,M796,Smärtor i extremitet,2017-04-10 09:43:39
2,K1055061,10224222,Akutkliniken Läk,Huvuddiagnos,Slutenvård,R074,"Bröstsmärtor, ospecificerade",2017-03-14 09:34:43
3,K1000309,10088130,Akutkliniken Läk,Huvuddiagnos,Slutenvård,R519,Huvudvärk,2017-03-10 07:59:32
4,K1000341,10024706,Akutkliniken Läk,Huvuddiagnos,Slutenvård,A469,Rosfeber,2017-01-01 14:07:07


## Triage

In [9]:
triage_df = pd.read_csv(FILES["PATIENTLIGGAREN_Vårdkontakter_2017_2018_Triagekategorier.csv"], sep="|", encoding="ISO-8859-1")
triage_df = triage_df.sort_values("Handelse_HandelseDatum").drop_duplicates("KontaktId").drop("PrioritetId", axis=1).rename(columns={"Handelse_HandelseDatum": "Triage_Timestamp"})

## Labb

In [10]:
labb = pd.read_csv(FILES["MELIOR_LabanalyserInom24TimmarFrånAnkomst.csv"], sep="|", encoding="ISO-8859-1")
labb["Analyssvar_ProvtagningDatum"] = pd.to_datetime(labb["Analyssvar_ProvtagningDatum"])
labb.head()

/tmp/ipykernel_16351/3018889116.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  labb = pd.read_csv(FILES["MELIOR_LabanalyserInom24TimmarFrånAnkomst.csv"], sep="|", encoding="ISO-8859-1")


,KontaktId,Alias,Term_Namn,Labanalys_Namn,Labanalys_AnalysTyp,Labanalys_BestallningTyp,Labanalys_Beskrivning,Analyssvar_Varde,Analyssvar_Enhet,Analyssvar_Attribut,Analyssvar_Referensvarde,Analyssvar_ReferensvardeMin,Analyssvar_ReferensvardeMax,Analyssvar_ProvtagningDatum
0,K1581100,10041182,Kemlab analys,P-Standard,A,7,P-Standardbikarbonat,26,mmol/L,0,3,22.0,27.0,2018-10-30 17:40:00
1,K1580269,10002978,Kemlab analys,B-Trombocy,A,7,B-Trombocyter,245,10^9/L,0,3,165.0,387.0,2018-10-29 19:50:00
2,K1581100,10041182,Kemlab analys,U-Protein,A,7,U-Protein (remsa),0,NaN,0,3,NaN,NaN,2018-10-30 17:40:00
3,K1580588,10128524,Kemlab analys,P-Standard,A,7,P-Standardbikarbonat,24,mmol/L,0,3,22.0,27.0,2018-10-30 10:30:00
4,K1580588,10128524,Kemlab analys,B-Trombocy,A,7,B-Trombocyter,KOMM,10^9/L,0,0,NaN,NaN,2018-10-30 09:30:00


In [ ]:
list(labb)

In [ ]:
l = labb.drop("KontaktId", axis=1).drop_duplicates()

In [ ]:
for k, v in l["Labanalys_Beskrivning"].value_counts().items():
    units_vc = l[l["Labanalys_Beskrivning"] == k]["Analyssvar_Enhet"].value_counts()
    units_vc = sorted(units_vc.items(),key=lambda x:-x[1])
    na_unit = l[l["Labanalys_Beskrivning"] == k]["Analyssvar_Enhet"].isna().sum()
    unit_s = "\t".join([f'{a}: {b}' for a,b in units_vc])
    if na_unit:
        unit_s += f'\t---MISSING--- ({na_unit})'
    print(f'{k}\t{v}\t{unit_s}')

In [ ]:
labb["Labanalys_Beskrivning"].value_counts()

In [ ]:
print(set(labb["KontaktId"]).issubset(liggar_kontaktid))
print(set(labb["Alias"]).issubset(liggar_alias))


## Melior diagnoser

In [11]:
melior_diagnoser_5ar_fore = pd.read_csv(FILES["MELIOR_Diagnoser5ÅrFöreVårdkontakt_2017_2018.csv"], sep="|", encoding="ISO-8859-1")
melior_diagnoser_vid_kontakt = pd.read_csv(FILES["MELIOR_DiagnoserVidVårdkontakt_2017_2018.csv"], sep="|", encoding="ISO-8859-1")
melior_diagnoser_30dagar_efter = pd.read_csv(FILES["MELIOR_Diagnoser30DagarEfterVårdkontakt_2017_2018.csv"], sep="|", encoding="ISO-8859-1")

In [11]:
print(set(melior_diagnoser_5ar_fore["KontaktId"]).issubset(liggar_kontaktid))
print(set(melior_diagnoser_5ar_fore["Alias"]).issubset(liggar_alias))

NameError: name 'liggar_kontaktid' is not defined

### Melior saknar startdatum

In [12]:
for df in [melior_diagnoser_5ar_fore, melior_diagnoser_vid_kontakt, melior_diagnoser_30dagar_efter]:
    l_before = len(df)
    blank = df["VårdtillfälleFörDiagnos_StartDatum"].isna().sum()
    df.dropna(subset=["VårdtillfälleFörDiagnos_StartDatum"], inplace=True)
    df["VårdtillfälleFörDiagnos_StartDatum"] = pd.to_datetime(df["VårdtillfälleFörDiagnos_StartDatum"])
    l_after = len(df)
    print(f'{l_before}\t{blank}\t{l_after}')
    

23631961	235	23631726
1466052	0	1466052
1064804	69	1064735


### Browsing

In [ ]:
a = melior_diagnoser_30dagar_efter
b = melior_diagnoser_vid_kontakt
c = melior_diagnoser_5ar_fore

b[b["Alias"] == 10091994]

In [ ]:
display(list(melior_diagnoser_30dagar_efter))
display(list(melior_diagnoser_vid_kontakt))
display(list(melior_diagnoser_5ar_fore))

a = set(list(melior_diagnoser_30dagar_efter))
b = set(list(melior_diagnoser_vid_kontakt))
c = set(list(melior_diagnoser_5ar_fore))
print(a-b)

In [ ]:
melior_diagnoser_5ar_fore["TermId"].value_counts()

In [ ]:
melior_diagnoser_5ar_fore['Diagnostyp'].value_counts()

In [ ]:
melior_diagnoser_5ar_fore["AktivitetTyp"].value_counts()

In [ ]:
melior_diagnoser_5ar_fore[["Diagnostyp","TermId"]].value_counts()

## Melior Lakemedel

In [ ]:
# Don't think we need this?

#melior_lakemedel_pre = pd.read_csv(FILES["MELIOR_OrdinationerSamtUtdelningarEttÅrFöreAnkomst.csv"], sep="|", encoding="ISO-8859-1", quoting=3)

In [ ]:
#melior_lakemedel_pre.head(1).T

## Socialstyrelsen

### Koder socialstyrelsen

In [13]:
hospital_map = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/resources/socialstyrelsen_hospital_map.csv").set_index("code")
mvo_map = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/resources/socialstyrelsen_mvo_map.csv").set_index("code")
lk_map = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/resources/scb_lk_koder.csv").set_index("code")
sos_kva_df = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/resources/sos_patientregister_kva/kva-inkl-beskrivningstexter-2021-rev2020-12-23.KVÅ-alla.csv").set_index("KOD")
sectra_map = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/resources/sectra_koder/Undersöknings koder RIS-AB.csv").set_index("Kod")

### SoS nyckel

In [14]:
sos_nyckel = pd.read_csv(FILES["SCB_Ekelund_LEV_Nyckel.csv"], sep="|", encoding="ISO-8859-1")
display(list(sos_nyckel))
display(sos_nyckel.head())
display(len(sos_nyckel))

['Alias', 'LopNr', 'AterPNr', 'SenPNr', 'SamOrdnNr', 'FelPersonnr']

,Alias,LopNr,AterPNr,SenPNr,SamOrdnNr,FelPersonnr
0,10000004,207785,0,1,0,0
1,10000008,63827,0,1,0,0
2,10000012,142319,0,1,0,0
3,10000016,175408,0,1,0,0
4,10000020,21680,0,1,0,0


317071

In [18]:
sos_nyckel["LopNr"].value_counts().sort_values()

305775    1
189350    1
301107    1
19365     1
120134    1
         ..
82385     1
183638    1
211353    1
160151    2
103968    2
Name: LopNr, Length: 317069, dtype: int64

### Join SoS LopNr i Liggaren

In [19]:
#liggaren = liggaren.merge(sos_nyckel[["Alias","LopNr"]], on="Alias", suffixes=("","_SoS"), how="left")
#liggaren["LopNr"] = liggaren["LopNr"].fillna(-1).astype(int)
#liggaren["LopNr"].value_counts()

## Behövs ej?

### SoS Sluten

In [15]:
sos_sluten = pd.read_csv(FILES["SOS_T_T_T_R_PAR_SV_24129_2020.csv"], sep="|", encoding="ISO-8859-1")
#sos_sluten["LopNr"] = sos_sluten["LopNr"].astype(int)
sos_sluten["INDATUM"] = pd.to_datetime(sos_sluten["INDATUM"])
sos_sluten["UTDATUM"] = pd.to_datetime(sos_sluten["UTDATUM"])
sos_sluten = sos_sluten.merge(sos_nyckel[["Alias","LopNr"]], on="LopNr", suffixes=("_SV","_Nyckel"), how="left")
sos_sluten["Alias"] = sos_sluten["Alias"].fillna(-1).astype(int)
sos_sluten["SJUKHUS"] = sos_sluten["SJUKHUS"].astype(str).replace(hospital_map.hospital)
sos_sluten["MVO"] = sos_sluten["MVO"].fillna("-1").astype(int).replace(mvo_map.MVO)
display(list(sos_sluten))
display(sos_sluten.head())

/tmp/ipykernel_16351/772931258.py:1: DtypeWarning: Columns (2,3,4,12,39,40,41,42,43,44,45,46,47,48,49,50,51,52,61,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  sos_sluten = pd.read_csv(FILES["SOS_T_T_T_R_PAR_SV_24129_2020.csv"], sep="|", encoding="ISO-8859-1")


['EKOD1',
 'EKOD2',
 'EKOD3',
 'EKOD4',
 'EKOD5',
 'hdia',
 'LopNr',
 'AR',
 'LK',
 'FODDAT',
 'KON',
 'ALDER',
 'SJUKHUS',
 'MVO',
 'INDATUMA',
 'UTDATUMA',
 'INDATUM',
 'UTDATUM',
 'VTID',
 'INSATT',
 'UTSATT',
 'PVARD',
 'OP',
 'OPD1',
 'OPD2',
 'OPD3',
 'OPD4',
 'OPD5',
 'OPD6',
 'OPD7',
 'OPD8',
 'OPD9',
 'OPD10',
 'OPD11',
 'OPD12',
 'OPD13',
 'OPD14',
 'OPD15',
 'OPD16',
 'OPD17',
 'OPD18',
 'OPD19',
 'OPD20',
 'OPD21',
 'OPD22',
 'OPD23',
 'OPD24',
 'OPD25',
 'OPD26',
 'OPD27',
 'OPD28',
 'OPD29',
 'OPD30',
 'DIA_ANT',
 'OP_ANT',
 'FLAND',
 'SENINV',
 'SENUTV',
 'CIVIL',
 'NATION',
 'LKF',
 'LKF_IN',
 'DIA1',
 'DIA2',
 'DIA3',
 'DIA4',
 'DIA5',
 'DIA6',
 'DIA7',
 'DIA8',
 'DIA9',
 'DIA10',
 'DIA11',
 'DIA12',
 'DIA13',
 'DIA14',
 'DIA15',
 'DIA16',
 'DIA17',
 'DIA18',
 'DIA19',
 'DIA20',
 'DIA21',
 'DIA22',
 'DIA23',
 'DIA24',
 'DIA25',
 'DIA26',
 'DIA27',
 'DIA28',
 'DIA29',
 'DIA30',
 'Alias']

,EKOD1,EKOD2,EKOD3,EKOD4,EKOD5,hdia,LopNr,AR,LK,FODDAT,...,DIA22,DIA23,DIA24,DIA25,DIA26,DIA27,DIA28,DIA29,DIA30,Alias
0,NaN,NaN,NaN,NaN,NaN,O80-O84,315247,2019,2084.0,198605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10217560
1,NaN,NaN,NaN,NaN,NaN,K859,266616,2019,1281.0,197202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10242665
2,NaN,NaN,NaN,NaN,NaN,I742,97825,2019,1231.0,193512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10249389
3,NaN,NaN,NaN,NaN,NaN,NaN,164241,2019,1230.0,199909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10143691
4,Y05,NaN,NaN,NaN,NaN,NaN,164241,2019,1230.0,199909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10143691


In [ ]:
sos_sluten["MVO"].value_counts()

### SoS Oppen

In [16]:
sos_oppen = pd.read_csv(FILES["SOS_T_T_T_R_PAR_OV_24129_2020.csv"], sep="|", encoding="ISO-8859-1")
sos_oppen["INDATUM"] = pd.to_datetime(sos_oppen["INDATUM"])
sos_oppen = sos_oppen.merge(sos_nyckel[["Alias","LopNr"]], on="LopNr", suffixes=("_OV","_Nyckel"), how="left")
sos_oppen["Alias"] = sos_oppen["Alias"].fillna(-1).astype(int)
sos_oppen["SJUKHUS"] = sos_oppen["SJUKHUS"].astype(str).replace(hospital_map.hospital)
#sos_oppen["MVO"] = sos_oppen["MVO"].fillna("-1").astype(int).replace(mvo_map.MVO)  ## XXX: Doesn't work due to 'Flo' in the column, whatever the hell that is. Hospital is always '30439', but it's not all the cases of that hospital...
display(list(sos_oppen))
display(sos_oppen.head())

/tmp/ipykernel_16351/1714827133.py:1: DtypeWarning: Columns (1,2,3,4,14,15,19,20,21,22,33,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  sos_oppen = pd.read_csv(FILES["SOS_T_T_T_R_PAR_OV_24129_2020.csv"], sep="|", encoding="ISO-8859-1")


['EKOD1',
 'EKOD2',
 'EKOD3',
 'EKOD4',
 'EKOD5',
 'EKOD6',
 'EKOD7',
 'hdia',
 'LopNr',
 'AR',
 'LK',
 'FODDAT',
 'KON',
 'ALDER',
 'SJUKHUS',
 'MVO',
 'INDATUMA',
 'INDATUM',
 'PVARD',
 'KTYP',
 'IN_AKUT_TIDPUNKT',
 'BED_AKUT_TIDPUNKT',
 'UT_AKUT_TIDPUNKT',
 'VERKS_AKUT',
 'OP',
 'DIA_ANT',
 'OP_ANT',
 'FLAND',
 'SENINV',
 'SENUTV',
 'CIVIL',
 'NATION',
 'LKF',
 'LKF_IN',
 'DIA1',
 'DIA2',
 'DIA3',
 'DIA4',
 'DIA5',
 'DIA6',
 'DIA7',
 'DIA8',
 'DIA9',
 'DIA10',
 'DIA11',
 'DIA12',
 'DIA13',
 'DIA14',
 'DIA15',
 'DIA16',
 'DIA17',
 'DIA18',
 'DIA19',
 'DIA20',
 'DIA21',
 'DIA22',
 'DIA23',
 'DIA24',
 'DIA25',
 'DIA26',
 'DIA27',
 'DIA28',
 'DIA29',
 'DIA30',
 'Alias']

,EKOD1,EKOD2,EKOD3,EKOD4,EKOD5,EKOD6,EKOD7,hdia,LopNr,AR,...,DIA22,DIA23,DIA24,DIA25,DIA26,DIA27,DIA28,DIA29,DIA30,Alias
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209613,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10152901
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J189,214097,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10242831
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264659,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10212494
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264659,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10212494
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264659,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10212494


In [ ]:
## Ignore this for now, but dunno what Flo is (see also above, MVO)

pd.set_option('display.max_rows', 50)
#sos_oppen[(sos_oppen["MVO"] == "Flo")].head().T
sos_oppen["SJUKHUS"].value_counts()
#sos_oppen.head().T

### SoS DORS

In [17]:
sos_dors = pd.read_csv(FILES["SOS_R_DORS_24129_2020.csv"], sep="|", encoding="ISO-8859-1")
sos_dors = sos_dors.merge(sos_nyckel[["Alias","LopNr"]], on="LopNr", suffixes=("_DORS","_Nyckel"), how="left")
sos_dors["Alias"] = sos_dors["Alias"].fillna(-1).astype(int)
sos_dors["LK"] = sos_dors["LK"].fillna("-1").astype(int).replace(lk_map.name)
sos_dors["DOD_KOMMUN"] = sos_dors["DOD_KOMMUN"].fillna("-1").astype(int).replace(lk_map.name)
#sos_dors["DODSDAT"] = pd.to_datetime(sos_dors["DODSDAT"], format="%Y%m%d")  # Doesn't work due to unclear death dates
display(list(sos_dors))
display(sos_dors.head())

/tmp/ipykernel_16351/3181732963.py:1: DtypeWarning: Columns (27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  sos_dors = pd.read_csv(FILES["SOS_R_DORS_24129_2020.csv"], sep="|", encoding="ISO-8859-1")


['LopNr',
 'AR',
 'DODSDAT',
 'KON',
 'alder',
 'LK',
 'ICD',
 'LKF',
 'ULORSAK',
 'KAP19',
 'MORSAK1',
 'MORSAK2',
 'MORSAK3',
 'MORSAK4',
 'MORSAK5',
 'MORSAK6',
 'MORSAK7',
 'MORSAK8',
 'MORSAK9',
 'MORSAK10',
 'MORSAK11',
 'MORSAK12',
 'MORSAK13',
 'MORSAK14',
 'MORSAK15',
 'MORSAK16',
 'MORSAK17',
 'MORSAK18',
 'MORSAK19',
 'MORSAK20',
 'MORSAK21',
 'MORSAK22',
 'MORSAK23',
 'MORSAK24',
 'MORSAK25',
 'MORSAK26',
 'MORSAK27',
 'MORSAK28',
 'MORSAK29',
 'MORSAK30',
 'MORSAK31',
 'MORSAK32',
 'MORSAK33',
 'MORSAK34',
 'MORSAK35',
 'MORSAK36',
 'MORSAK37',
 'MORSAK38',
 'MORSAK39',
 'MORSAK40',
 'MORSAK41',
 'MORSAK42',
 'MORSAK43',
 'MORSAK44',
 'MORSAK45',
 'MORSAK46',
 'MORSAK47',
 'MORSAK48',
 'DALDKL5',
 'DBGRUND1',
 'DBGRUND5',
 'DBGRUND6',
 'ALKOHOL',
 'DIABETES',
 'NATION',
 'FODLAND',
 'DODSPL',
 'DODUTL',
 'OPERERAD',
 'FOBUTL',
 'NATION_FOBUTL',
 'DOD_KOMMUN',
 'FODDATN',
 'Alias']

,LopNr,AR,DODSDAT,KON,alder,LK,ICD,LKF,ULORSAK,KAP19,...,NATION,FODLAND,DODSPL,DODUTL,OPERERAD,FOBUTL,NATION_FOBUTL,DOD_KOMMUN,FODDATN,Alias
0,185581,2018,20180629,2,106,Malmö,10,NaN,I209,NaN,...,Sverige,Sverige,2.0,NaN,2.0,NaN,Uppgift Saknas,Malmö,1911-07,10093991
1,46635,2017,20170529,2,105,Kristianstad,10,NaN,I219,NaN,...,Sverige,Sverige,1.0,NaN,2.0,NaN,Uppgift Saknas,-1,1911-07,10208086
2,44511,2018,20180301,2,105,Helsingborg,10,NaN,J189,NaN,...,Sverige,Sverige,2.0,NaN,2.0,NaN,Uppgift Saknas,Helsingborg,1912-05,10304166
3,89151,2017,20170210,1,104,Vellinge,10,NaN,I619,NaN,...,Sverige,Sverige,3.0,NaN,2.0,NaN,Uppgift Saknas,Vellinge,1912-07,10134952
4,57093,2017,20171109,1,105,Malmö,10,NaN,I442,NaN,...,Sverige,Sverige,1.0,NaN,2.0,NaN,Uppgift Saknas,Malmö,1912-07,10262556


In [ ]:
x = sos_dors[sos_dors["Alias"] == 100939911]
if len(x) > 0:
    display(x)

In [ ]:
pd.to_datetime(sos_dors["DODSDAT"], errors="coerce", format="%Y%m%d").dropna().hist(bins=24, figsize=(10,10))

### SoS Lakemedel

In [18]:
sos_lakemedel = pd.read_csv(FILES["SOS_T_R_LMED_24129_2020.csv"], sep="|", encoding="ISO-8859-1")
sos_lakemedel = sos_lakemedel.merge(sos_nyckel[["Alias","LopNr"]], on="LopNr", suffixes=("_LM","_Nyckel"), how="left")
sos_lakemedel["Alias"] = sos_lakemedel["Alias"].fillna(-1).astype(int)
sos_lakemedel["EDATUM"] = pd.to_datetime(sos_lakemedel["EDATUM"])

In [ ]:
sos_lakemedel.dtypes

### ATC och ICD koder från SoS

In [ ]:
dia = ["hdia"] + [f"DIA{i}" for i in range(1,31)]
sos_sluten[dia].stack().value_counts().to_csv("sos_sluten_hdia+DIA1-DIA30_ICD_list.csv")
sos_sluten["hdia"].value_counts().to_csv("sos_sluten_hdia_ICD_list.csv")

In [ ]:
dia = ["hdia"] + [f"DIA{i}" for i in range(1,31)]
sos_oppen[dia].stack().value_counts().to_csv("sos_oppen_hdia+DIA1-DIA30_ICD_list.csv")
sos_oppen["hdia"].value_counts().to_csv("sos_oppen_hdia_ICD_list.csv")

In [ ]:
# Trivial dump of KVA, superseeded by the code further below.

#df = pd.DataFrame()
#df["KVA"] = [kva for opstr in list(sos_sluten["OP"].fillna("")) for kva in opstr.split(" ")]
#df.value_counts().to_csv("sos_sluten_OP_list.csv")

In [ ]:
vc = sos_lakemedel["ATC"].value_counts()
vc.to_csv("sos_lakemedel_ATC_list.csv")
vc.sort_index().to_csv("sos_lakemedel_ATC_list_sorted.csv")

In [ ]:
# Dump KVA with lookup

i=0
c = Counter()

for a, df in zip(["SV", "OV"],[sos_sluten,sos_oppen]):
    for s in list(df["OP"]):
        i+=1
        if not pd.isna(s):
            for q in s.split(" "):
#                print(q)
                c[q] += 1
#        if i == 200:
#            break
    
    print(f"====== {a} ======")
    codes = []
    counts = []
    for k in sorted(c.keys(),key=lambda x:c[x], reverse=True):
        print(f"{k}\t{c[k]}")
        codes.append(k)
        counts.append(c[k])
    out_df = pd.DataFrame(data={'KOD': codes, 'count': counts}).set_index("KOD")
    out_df = out_df.join(sos_kva_df)
    out_df["KLASSDEL"] = out_df["KLASSDEL"].fillna(-1).astype(int)
    out_df.to_csv(f"skane1718_soos_{a}_kva.counts.csv", encoding="UTF8")
    out_df.sort_index().to_csv(f"skane1718_soos_{a}_kva_sorted.counts.csv", encoding="UTF8")
    

## SCB

In [19]:
a = pd.read_csv(FILES["SCB_Ekelund_LEV_LISA2017.csv"], sep="|", encoding="ISO-8859-1")

In [361]:
len(set(a["lopnr"]))

306949

In [368]:
a["Kommun"].isna().sum()

0

In [366]:
sorted(a["Lan"].value_counts().keys())

[1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25]

In [360]:
a.iloc[0,:]

lopnr                   242173
SenPNr                       1
AterPNr                      0
SamOrdnNr                    0
FelPersonnr                  0
FodelseAr                 1981
Kon                          1
Lan                         12
Kommun                    1290
Distriktskod          101248.0
Civil                        G
MedbGrEg                     0
MedbGrEg4                    0
Barn0_3                      1
Barn4_6                      1
Barn7_10                     0
Barn11_15                    0
Barn16_17                    0
Sun2000niva_old              3
Sun2000niva                322
Sun2000Inr                010b
ExamAr                    2001
SyssStat11                   1
YrkStalln                    2
Ssyk4_2012_J16            2149
SsykAr_J16                2017
SsykStatus_J16             1.0
ESeG_J16                   2.1
ForLed                      40
SjukPP                       0
SjukTyp                      0
SjukRe                       0
SjukP_Nd

## Sectra

In [20]:
sectra = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/data/Skane-1718-SECTRA-3rd/20211104/SECTRA_alla_2012-2018_flik1-3_20211104.csv", sep="|", encoding="ISO-8859-1")
sectra19 = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/data/Skane-1718-SECTRA-2019/20211112/SECTRA_2019_20211112.csv", sep="|", encoding="ISO-8859-1")
sectra = pd.concat([sectra,sectra19])
sectra = sectra.reset_index(drop=True)
sectra["Undersökningsdatum"] = pd.to_datetime(sectra["Undersökningsdatum"])
sectra["Datum inskickad remiss"] = pd.to_datetime(sectra["Datum inskickad remiss"])
sectra = sectra.join(sectra_map, on="Undersökningskod")
sectra["Typ"] = sectra["Typ"].fillna("---Saknas---")
sectra["Undersökningsnamn"] = sectra["Undersökningsnamn"].fillna("---Saknas---")

In [ ]:
((sectra["Undersökningskod"] == "68600") &
(sectra["Undersökningsdatum"] > datetime(year=2017,month=1,day=1))).sum()

## Generate HTML

In [29]:
importlib.reload(skane1718)
import mim.extractors.skane_1718 as skane1718

dfs = {
    "liggaren": liggaren,
    "prelim": prelim,
    "labb": labb,
    "melior_kontakt": melior_diagnoser_vid_kontakt,
    "melior_post_30dagar": melior_diagnoser_30dagar_efter,
    "melior_pre_5yr": melior_diagnoser_5ar_fore,
    "sos_sluten": sos_sluten,
    "sos_oppen": sos_oppen,
    "sos_dors": sos_dors,
    "sos_lm": sos_lakemedel
}
            
    
### XXX: Checkout 10001265 -- nothing in Melior???
    
#10237978
#10227673 -- duplicate blood samples ???
alias = 10001558
skane1718.write_patient_html(alias, "foo.html", dfs)

In [28]:
melior_diagnoser_vid_kontakt["VårdtillfälleFörDiagnos_StartDatum"].astype(str)

0          2014-02-12 15:27:55
1          2014-02-12 15:27:55
2          2014-02-12 15:27:55
3          2014-02-12 15:27:55
4          2014-02-12 15:27:55
                  ...         
1466047    2018-12-21 10:40:17
1466048    2018-12-25 22:01:00
1466049    2018-12-29 01:11:35
1466050    2018-12-30 18:25:00
1466051    2017-02-01 13:23:22
Name: VårdtillfälleFörDiagnos_StartDatum, Length: 1466052, dtype: object

In [ ]:
HTML_ROOT="/mnt/air-crypt/air-crypt-esc-trop/andersb/temp-skane-1718/pat_html/"

count = 0
for alias in tqdm(liggar_alias):
#    print(alias)
    try:
        file =  HTML_ROOT + str(alias) + ".html"
        if not os.path.exists(file):
            skane1718.write_patient_html(alias,file, dfs)
        count += 1
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print("Problem with "+str(alias))
        print(e)
        print(traceback.format_exc())
        print()
#    if count == 100:
#        print("Breaking!")
#        break

## SVAR

In [191]:
svar = pd.read_csv(FILES["SVAR_Hela_Skåne_17_18.csv"], sep="|", encoding="ISO-8859-1")
display(list(svar))
display(svar.head())
display(len(svar))

['CentridTRIMJUST',
 'centreidAkutvårdsregistrering',
 'BesöksIDAkutvårdsregistrering',
 'Födelsedatum',
 'Åldervidankomsttillakutmottagning',
 'Identitetstyp',
 'Dödsdatumbefregkontrollerat',
 'AnkomsttillakutmottagningAkutvårdsregistrering',
 'Ankomstdatumtillakutmottagning',
 'PlaneratåterbesökAkutvårdsregistrering',
 'AnkomstsättAkutvårdsregistrering',
 'AnkommenfrånAkutvårdsregistrering',
 'AnkomstlarmAkutvårdsregistrering',
 'RemissAkutvårdsregistrering',
 'remiteradfranAkutvårdsregistrering',
 'BesöksorsakAkutvårdsregistrering',
 'HänvisningAkutvårdsregistrering',
 'TidsstämpelVitalparameter',
 'SystolisktblodtryckVitalparameter',
 'DiastolisktblodtryckVitalparameter',
 'Hjärtfrekvens Vitalparameter',
 'Andningsfrekvens Vitalparameter',
 'SaturationVitalparameter',
 'Kroppstemperatur Vitalparameter',
 'MedvetandegradRLSVitalparameter',
 'MedvetandegradAVPUVitalparameter',
 'MedvetandegradenligtGlascowComaScale Vitalparameter',
 'TriagenivåAkutvårdsregistrering',
 'Sjukskötersket

,CentridTRIMJUST,centreidAkutvårdsregistrering,BesöksIDAkutvårdsregistrering,Födelsedatum,Åldervidankomsttillakutmottagning,Identitetstyp,Dödsdatumbefregkontrollerat,AnkomsttillakutmottagningAkutvårdsregistrering,Ankomstdatumtillakutmottagning,PlaneratåterbesökAkutvårdsregistrering,...,Ankomstmånad,Ankomstvecka,Ankomstdagtxt,Åldersklass,Åldersklass20års,filter_$,strdate,TTLklass,TVTklass,Region
0,Ystad,Ystad,1147638-1,25-jul-52,64.0,Kvinna,#NULL!,2017-06-26 18:11,26-jun-17,Nej,...,6.0,26.0,måndag,70.0,80.0,1.0,NaN,"0,5-1",>12,Skåne
1,Ystad,Ystad,1087505-9,29-aug-16,0.0,Kvinna,#NULL!,2017-06-26 18:10,26-jun-17,Nej,...,6.0,26.0,måndag,10.0,20.0,1.0,NaN,NaN,Under 2,Skåne
2,Ystad,Ystad,1147645-1,01-dec-30,86.0,Man,#NULL!,2017-06-26 18:07,26-jun-17,Nej,...,6.0,26.0,måndag,90.0,100.0,1.0,NaN,>6,6-8,Skåne
3,Ystad,Ystad,796376-7,23-aug-34,82.0,Man,#NULL!,2017-06-26 17:55,26-jun-17,Nej,...,6.0,26.0,måndag,90.0,100.0,1.0,NaN,"1,5-2",10-12,Skåne
4,Ystad,Ystad,208621-8,30-maj-30,87.0,Man,#NULL!,2017-06-26 17:50,26-jun-17,Nej,...,6.0,26.0,måndag,90.0,100.0,1.0,NaN,"0-0,5",6-8,Skåne


702551

In [ ]:
pd.to_datetime(svar["AnkomsttillakutmottagningAkutvårdsregistrering"]).isna().sum()

In [ ]:
svar = svar.dropna(how="all")

In [ ]:
liggaren["Vardkontakt_InskrivningDatum"].sort_values()

In [ ]:
svar["AnkomsttillakutmottagningAkutvårdsregistrering"].sort_values()

In [ ]:
list(liggaren)

In [ ]:
AktuellSjukhudId_2_Sjukhus_Namn = {
    "SE162321000255-O11414": "SUS Malmö",
    "SE162321000255-O12014": "Helsingborgs lasarett",
    "O10001": "SUS Lund",
    "SE162321000255-O10532": "Centralsjukhuset Kristianstad",
    "SE162321000255-O16571": "Ystads lasarett",
    "SE162321000255-O11016": "Lasarettet Trelleborg",
    "O11249": "Hässleholms sjukhus",
    "O10971": "Lasarettet i Landskrona",
    "SE162321000255-O12784": np.nan,
    "SE162321000255-O22178YLOE": np.nan,
    "SE162321000255-O11962": "Ängelholms sjukhus",
    "SE162321000255-O18814KROE": np.nan,
    "O12984": "Barn- och ungdomsmedicinska kliniken Malmö",
    "O17830KR": "Psykiatri Skåne Kristianstad",
    "SE162321000255-O20919": "Kvälls- och Helgmottagningen Södervärn Malmö"
}

In [ ]:
liggaren[liggaren["AktuellSjukhusId"] == "SE162321000255-O18814KROE"]

In [ ]:
s = "SE162321000255-O20919"
x = liggaren[liggaren["AktuellSjukhusId"] == s]["Sjukhus_Namn"]
display(len(x))
display(x.isna().sum())
display(x.value_counts())

In [ ]:
liggaren["AktuellSjukhusId"].value_counts()

In [ ]:
x_key_cols=[
    "Vardkontakt_InskrivningDatum",
    "Vardkontakt_UtskrivningDatum",
    "Vardkontakt_PatientAlderVidInskrivning",
    "Patient_Kon",
    "Sjukhus_Namn",
    "AktuellSjukhusId",
    "Inlagd"
]
for c in x_key_cols:
    display(f"{c} -- {liggaren[c].isna().sum()}")

In [ ]:
l = liggaren["Vardkontakt_InskrivningDatum"].value_counts()
l[l<2]


In [ ]:
l = set(pd.to_datetime(liggaren.drop_duplicates("Vardkontakt_InskrivningDatum", keep=False)["Vardkontakt_InskrivningDatum"]).sort_values())

In [ ]:
s = set(pd.to_datetime(svar.drop_duplicates("AnkomsttillakutmottagningAkutvårdsregistrering", keep=False)["AnkomsttillakutmottagningAkutvårdsregistrering"]).sort_values())

In [ ]:
len(l.intersection(s))

In [ ]:
len(liggaren)

In [ ]:
svar.iloc[38429,:]

In [ ]:
svar.iloc[563484,:]

## Bodil och Anders

### Besoksorsaker

In [ ]:
liggaren["BesokOrsak_Beskrivning"].value_counts()

In [ ]:
liggaren["BesokOrsak_Kod"].value_counts()

In [ ]:
len(liggaren)

In [ ]:
counts = (liggaren["BesokOrsak_Kod"].fillna("--SAKNAS--") + "~" + liggaren["BesokOrsak_Beskrivning"].fillna("--SAKNAS--")).value_counts()
for k in sorted(counts.keys()):
    print(f"{k}~{counts[k]}")

### Make df

In [ ]:
orsaker = ["Buksmärta RETTS-A 6, RETTS-P 106", "Smärta i extremitet RETTS-A 15, RETTS-P115"]
abexpain_liggare = liggaren[((liggaren["BesokOrsak_Beskrivning"] == orsaker[0]) | (liggaren["BesokOrsak_Beskrivning"] == orsaker[1]))]

In [ ]:
## I checked, for the 2 Socialstyrelsen Lopnr that map to two different Aliases, only one alias occurs in this subset, so we can safely make a join on Socialstyrelsens key here:
abexpain_liggare = abexpain_liggare.set_index("Alias").join(sos_nyckel.set_index("Alias"))

In [ ]:
abexpain_liggare["Vardkontakt_InskrivningDatum"] = pd.to_datetime(abexpain_liggare["Vardkontakt_InskrivningDatum"])
abexpain_liggare["LopNr"] = abexpain_liggare["LopNr"].fillna("-1").astype(int)

In [ ]:
abexpain_liggare.dtypes

### First glance

In [ ]:
abexpain_liggare.iloc[20:24].T

In [ ]:
## Initial info

print("=========")
display(f"Total number of visits: {len(abexpain_liggare)}. Män: {(abexpain_liggare['Patient_Kon'] =='M').sum()}, Kvinnor: {(abexpain_liggare['Patient_Kon'] =='F').sum()}")
print("=========")
display("By patient")
display(abexpain_liggare.index.value_counts())
print("=========")
display(f"Inlagd: {abexpain_liggare['Inlagd'].sum()}. Män: {((abexpain_liggare['Patient_Kon'] =='M') & (abexpain_liggare['Inlagd'] == 1)).sum()}, Kvinnor: {((abexpain_liggare['Patient_Kon'] =='F') & (abexpain_liggare['Inlagd'] == 1)).sum()}")
print("=========")
display("Utskriven till")
display(abexpain_liggare["Utskriven till"].value_counts())
print("=========")
display("Sjukhus fördelning")
display(abexpain_liggare["Sjukhus_Namn"].fillna("??? Saknas ???").value_counts())
print("=========")

In [ ]:
## Eventuellt återanvända personnummer

abexpain_liggare["AterPNr"].sum()

In [ ]:
## Distribution över tid

pd.to_datetime(abexpain_liggare["Vardkontakt_InskrivningDatum"]).hist(bins=24, figsize=(10,10))

In [ ]:
abexpain_liggare["Vardkontakt_PatientAlderVidInskrivning"].hist(bins=20, figsize=(10,10))

In [ ]:
abexpain_liggare[abexpain_liggare["Utskriven till"] == "Avdelning"]["Inläggningsavdelning"].value_counts()

### Diagnoser

In [ ]:
#Arterial thrombosis in the abdominal or lower extremity circulation: I74.0, I74.1, I 74.2, I 74.3, I74.4, I74.5, I74.8, I74.9, K55.0, K55.1, K55.9, N28.0.
#Venous thrombosis in the abdominal or lower extremity circulation: I80.1, I80.2, I80.3, I80.8, I80.9, I81.9.
icd = set(sos_sluten["hdia"])


In [ ]:
art_thromb_icd = ["I740", "I741", "I742", "I743", "I744", "I745", "I748", "I749", "K550", "K559", "N280", "N28"]
ven_thromb_icd = ["I801", "I802", "I803", "I808", "I809", "I819"]
for i in art_thromb_icd:
    print(f"{i}\t{i in icd}")

#### Allmän överblick över huvuddiagnoser i slutenvårdsregistret från SoS.

In [ ]:
#for k,v in sos_sluten["hdia"].value_counts().items():
#    print(f'{k}\t{v}')
c = sos_sluten["hdia"].value_counts()
print("Förekomst av diagnoser i hela slutenvårdsregistret (även andra sökorsaker)")
display(c)


## N28* har trunkerats till N28.

for s, l in zip(["Arterial thrombosis", "Venuous thrombosis"], [art_thromb_icd, ven_thromb_icd]):
    print(s)
    for i in l:
        if i in c:
            print(f"{i}\t{c[i]}")
        else:
            print(f"{i} -- missing")
    print()

#### Tromboser i Slutenvården

In [ ]:
skipped = 0
acount = 0
vcount = 0
ecount = 0
for alias, r in abexpain_liggare.iterrows():
    lopnr = r["LopNr"]
    if lopnr == -1:
        skipped += 1
        continue
#    print(lopnr)
    v, a = False, False
    for hdia in sos_sluten[(sos_sluten["LopNr"] == lopnr) & (r["Vardkontakt_InskrivningDatum"] <= sos_sluten["INDATUM"]) & (sos_sluten["INDATUM"] <= (r["Vardkontakt_InskrivningDatum"] + pd.Timedelta("90D")))]["hdia"]:
        a = a or hdia in art_thromb_icd
        v = v or hdia in ven_thromb_icd
    if a:
        acount += 1
    if v:
        vcount += 1
    if a or v:
        ecount += 1

print(f"Skipped: {skipped}")
print(f"Arterial: {acount}")
print(f"Venuous: {vcount}")
print(f"Either: {ecount}")
print(f"Total: {len(abexpain_liggare)}")

In [ ]:
abexpain_liggare.dtypes

## DVT export

In [ ]:
DVT_ORSAKER = ["Smärta i extremitet RETTS-A 15, RETTS-P115", "Svullnad i extremitet RETTS-A 15, RETTS-P 115", "Extremitetsproblem RETTS-A 15"]
BLOOD_SAMPLES = [
    "P-Kreatinin (enz)",
    "B-Hemoglobin (Hb)",
    "P-CRP",
    "P-D-dimer",
    "B-Leukocyter",
    "B-Trombocyter"
]
BLOOD_SAMPLE_MAX_HOURS_AFTER_INDEX=24

ATC_PREFIXES = ["A10", "B01", 
                "C02", "C03", "C08", "C09", "C10", 
                "G03A", "G03C",
                "H02",
                "L01", "L02A", "L04"
               ]
SECTRA_CODES = [96800, 68600, 83780, 83980, 68700, 83080, 83981, 68259, 
                68248, 48046, 68258, 83781, 68213, 68240, 68200, 48025, 
                38346, 48048, 48000, 38348, 38300]
SECTRA_CODES = set(map(str, SECTRA_CODES))
SECTRA_DAYS = 30
ATC_DAYS_BEFORE_INDEX = 90
SURGERY_DAYS_BEFORE_INDEX = 90
PREV_DISEASE_ICD_PREFIXES = [f"I{i}" for i in chain(range(10,17), range(20,26), range(30,51), range(60,70), range(80,83))] + [f"M{i:02d}" for i in chain(range(4,15), range(30,37))] + [f"C{i:02d}" for i in range(0,97)] + [f"E{i:02d}" for i in range(8,14)] + ["F10"]
TARGET_OUTCOME_ICD_PREFIXES = ["I801", "I802", "I803", "I26"]   # XXX: What about the others that start with I80?

OUTCOME_DAYS_AFTER_INDEX = [0, 30, 90, 180, 365] # XXX: Maybe change 0 to 1 here
DVT_EXPORT_ROOT = "/mnt/air-crypt/air-crypt-esc-trop/andersb/temp-skane-1718/dvt-export"
LIGGARE_KEEP_COLUMNS=[
    "Vardkontakt_InskrivningDatum", "BesokOrsak_Kod",
    "Patient_Kon", "Vardkontakt_PatientAlderVidInskrivning",
    "Inlagd"
]


importlib.reload(s1718_util)
import mim.extractors.skane_1718_extract_util as s1718_util

t0 = time.time()

#INCLUDE_SOS_OV = True

CUT = 1000

dvt_liggare = liggaren[liggaren["BesokOrsak_Beskrivning"].isin(DVT_ORSAKER)].copy() # Select
dvt_liggare.loc[:,"Vardkontakt_InskrivningDatum"] = pd.to_datetime(dvt_liggare["Vardkontakt_InskrivningDatum"]) # Cast
dvt_liggare = dvt_liggare.sort_values("Vardkontakt_InskrivningDatum") # Sort chronologically
print(len(dvt_liggare))
dvt_liggare = dvt_liggare.drop_duplicates("Alias") # Reduce to first visit for each patient
print(len(dvt_liggare))
dvt_liggare = dvt_liggare.set_index("Alias").join(sos_nyckel.set_index("Alias")) # join SoS key
dvt_liggare = dvt_liggare[dvt_liggare["LopNr"].notna()]  # Drop 333 where there is no SoS entry.
dvt_liggare.loc[:,"LopNr"] = dvt_liggare["LopNr"].astype(int)  # Cast
print(len(dvt_liggare))
if CUT > 0:
    dvt_liggare = dvt_liggare.iloc[:CUT]
print(len(dvt_liggare))


prev_diseases_ov = s1718_util.extract_previous_diagnoses_sos_ov(sos_oppen, dvt_liggare.reset_index(),PREV_DISEASE_ICD_PREFIXES)
prev_diseases_sv = s1718_util.extract_previous_diagnoses_sos_sv(sos_sluten, dvt_liggare.reset_index(),
                                                      PREV_DISEASE_ICD_PREFIXES)
icd_outcomes_sv = s1718_util.extract_outcome_icds_sos(sos_sluten, dvt_liggare.reset_index(),
                                                  TARGET_OUTCOME_ICD_PREFIXES,
                                                  OUTCOME_DAYS_AFTER_INDEX,
                                                     "SV")
icd_outcomes_ov = s1718_util.extract_outcome_icds_sos(sos_oppen, dvt_liggare.reset_index(),
                                                  TARGET_OUTCOME_ICD_PREFIXES,
                                                  OUTCOME_DAYS_AFTER_INDEX,
                                                     "OV")
death = s1718_util.extract_death_sos_dors(sos_dors, dvt_liggare.reset_index(), 
                                         OUTCOME_DAYS_AFTER_INDEX)
sos_surgery_ov = s1718_util.extract_previous_surgery_sos_ov(sos_oppen, dvt_liggare.reset_index(),
                                                            SURGERY_DAYS_BEFORE_INDEX)
sos_surgery_sv = s1718_util.extract_previous_surgery_sos_sv(sos_sluten, dvt_liggare.reset_index(),
                                                           SURGERY_DAYS_BEFORE_INDEX)
print("LM")
lm = s1718_util.extract_historical_atc_sos(sos_lakemedel, dvt_liggare.reset_index(),
                                           ATC_DAYS_BEFORE_INDEX, ATC_PREFIXES)
lb = s1718_util.extract_index_blood_samples(labb, dvt_liggare.reset_index(),
                                            BLOOD_SAMPLES, BLOOD_SAMPLE_MAX_HOURS_AFTER_INDEX)
sec = s1718_util.extract_sectra_procedures(sectra, dvt_liggare.reset_index(),
                                           SECTRA_CODES, SECTRA_DAYS,
                                           1,
                                           None) #DVT_EXPORT_ROOT + "/to_annotate/"
out = dvt_liggare[LIGGARE_KEEP_COLUMNS]
out = out.join([prev_diseases_sv, prev_diseases_ov, icd_outcomes_sv, icd_outcomes_ov, death, sos_surgery_ov, sos_surgery_sv, sec, lm, lb])
out.to_csv("dvt-R.csv")

t1 = time.time()
print(f"Time: {t1-t0=}")

In [ ]:
sos_dors.DODSDAT

In [ ]:
sos_lakemedel["Alias"].isin(dvt_liggare.index).sum()

In [ ]:
#sos_lakemedel
d = datetime(year=2018,month=1, day=1) + timedelta(days=-90) + timedelta(days=-10)
print(len(sos_lakemedel))
print(len(sos_lakemedel[sos_lakemedel["EDATUM"] > d]))

In [ ]:
sos_lakemedel

In [ ]:
prev_diseases_sv

In [ ]:
BROKEN_BLOOD_SAMPLE_VALUES = [
    "FELTA", "HEMOL", "KOAG", "KOAGL", "KOMM", "OFYLL", "OTILL", "SAKNA"
]

new_data[~new_data["blood-P-D-Dimer"].isin(BROKEN_BLOOD_SAMPLE_VALUES)]

In [ ]:
new_data.dtypes

In [ ]:
list(dvt_liggare)

In [ ]:
importlib.reload(s1718_util)
import mim.extractors.skane_1718_extract_util as s1718_util

x = dvt_liggare["Vardkontakt_InskrivningDatum"].reset_index().rename(columns={"Vardkontakt_InskrivningDatum": "IndexVisitTimestamp"})
s1718_util.extract_historical_atc_sos(sos_lakemedel, x, ATC_DAYS_BEFORE_INDEX, ATC_PREFIXES)

In [ ]:
sectra.dtypes

In [ ]:
D=FAILED_SV_KVA_CODES

#for k in sorted(D.keys(), key=lambda x: (-D[x], x)):
#    print(f"{k} -- {D[k]}")
    
D=FIRING_SV_KVA_CODES
df = pd.DataFrame(data=np.array([(k, D[k]) for k in sorted(D.keys(), key=lambda x: (-D[x], x))]), columns=['KOD','count']).set_index("KOD")
df = df.join(sos_kva_df)
df.to_csv("firing_sv_kva.csv")

In [ ]:
for _, row in pat_sluten[(pat_sluten["INDATUM"] < index_date) & (pat_sluten["INDATUM"] > index_date - timedelta(days=(SURGERY_DAYS_BEFORE_INDEX+100)))][SV_OP_COLS].dropna(subset=["OP"]).iterrows():  # Add 100 days here to have some extra space backwards in time. We will consider the actual OP date below anyway
    ops = row["OP"].split(" ")
    for i, o in enumerate(ops, start=1):
        d = pd.to_datetime(row[i])
        print(f"{i}\t{o}\t{d}")
        if index_date-timedelta(days=SURGERY_DAYS_BEFORE_INDEX) < d < index_date:
            if o in sos_kva_df.index:
                print("yes")
            else:
                print("no")
        
        
print(index_date)
        




In [ ]:
for x in pat_oppen[(pat_oppen["INDATUM"] < index_date) & (pat_oppen["INDATUM"] > index_date - timedelta(days=SURGERY_DAYS_BEFORE_INDEX))].dropna(subset=["OP"])["OP"]:
    ops = x.split(" ")
    print(ops)
    for o in ops:
        if o in sos_kva_df.index:
            print(sos_kva_df.loc[o,:]["KLASSDEL"])
        else:
            print("Failed "+o)

In [ ]:
sos_kva_df.loc["XS100",:]

In [ ]:
new_data

In [ ]:
for icd in TARGET_OUTCOME_ICD_PREFIXES:
    print(f"{icd}\t{dvt_data[get_outcome_label(90,icd)].sum()}")

In [ ]:
dvt_data = dvt_liggare.join(new_data.set_index("Alias"))

def collapse_outcomes(days, row):
    cols = [get_outcome_label(days,icd) for icd in TARGET_OUTCOME_ICD_PREFIXES]
    if row[cols].sum():
        return 1
    else:
        return 0

for days in OUTCOME_DAYS_AFTER_INDEX:
    dvt_data[get_outcome_label(days, None)] = dvt_data.apply(partial(collapse_outcomes, days), axis=1)

In [ ]:
tot = len(dvt_data)
zero_d = dvt_data[get_outcome_label(0, None)].sum()
one80_d = dvt_data[get_outcome_label(90, None)].sum()
print(f"0d: {zero_d}/{tot} = {zero_d/tot}")
print(f"180d: {one80_d}/{tot} = {one80_d/tot}")

## Sectra counts

In [ ]:
dvt_sectra_365 = pd.read_csv("dvt-sectra-365.csv",names=["count","code"],header=None)
dvt_sectra_365

In [ ]:
dvt_sectra_365.join(sectra_map, on="code").to_csv("dvt-sectra-365.joined.csv")

In [ ]:
sectra_map

## Exempel

In [ ]:
sos_nyckel[sos_nyckel["Alias"] == 10261943]

In [ ]:
liggaren[liggaren["Alias"] == 10261943].T

In [ ]:
melior_diagnoser_vid_kontakt[melior_diagnoser_vid_kontakt["Alias"] == 10261943].T

In [ ]:
sos_sluten[sos_sluten["LopNr"] == 134374]

In [ ]:
abexpain_liggare.head(10)

In [ ]:
abexpain_liggare.loc[10000021]

In [ ]:
melior_diagnoser_vid_kontakt[melior_diagnoser_vid_kontakt["Alias"] == 10000021].T

In [ ]:
sos_nyckel[sos_nyckel["Alias"] == 10000021]

In [ ]:
for l in list(sos_sluten):
    print(l)

In [ ]:
sluten_cols=["EKOD1", "SJUKHUS", "MVO", "INDATUM", "UTDATUM", "hdia", "OP", "DIA_ANT", "DIA1", "DIA2", "DIA3", "DIA4"]

sos_sluten[sos_sluten["LopNr"] == 102295][sluten_cols].sort_values("INDATUM")

## Mismatch SoS -- Skåne17-18

In [ ]:
abexpain_liggare[abexpain_liggare['LopNr'].isna()]["Utskriven till"].value_counts()

## SECTRA

In [ ]:
sectra_a = pd.read_csv(FILES["SECTRA_export2012_2015A.csv"], sep="|", encoding="ISO-8859-1")
sectra_b = pd.read_csv(FILES["SECTRA_export2016_2018A.csv"], sep="|", encoding="ISO-8859-1")
sectra = pd.concat([sectra_a, sectra_b])

In [ ]:
sectra_a.dtypes

In [ ]:
sectra_a.sort_values("Undersökningsdatum")

In [ ]:
sectra_b.sort_values("Undersökningsdatum")

In [ ]:
## From 
## https://vardgivare.skane.se/siteassets/1.-vardriktlinjer/med-omraden/bild-funktion/undersokningsmetoder-bof-rontgen32.pdf
## and
## https://vardgivare.skane.se/siteassets/2.-patientadministration/avgifter-och-prislistor/prislistor/bfm/Prislista-bild-och-funktion.xlsx

sectra_koder = {
#     10000: "Neuroradiologisk us utan egen kod",
#     13000: "Lumbalpunktion",
#     13017: "Lumbalpunktion och tryckmätning",
#     13050: "Lumbalpunktion för odling/cellanalys",
#     13100: "Lumbothorakal myelografi",
#     13400: "Myelografi för efterföljande datortomografi",
#     13700: "Lumbal vertebroplastik",
#     17000: "Carotis communis-angiografi",
#     17048: "Carotis communis-angiografi med inläggning av stent",
#     17062: "Carotis communis-angiografi med stentgraft",
#     17130: "Carotis interna-angiografi med tomografi och/eller 3D",
#     17148: "Carotis interna-angiografi med inläggning av stent",
#     17162: "Carotis interna-angiografi med stentgraft",
#     17500: "Kombination av två eller flera skallangiografi",
#     17530: "Kombination av två eller flera skallangiografi med tomografi och/eller 3D",
#     17575: "Skallangiografi med intravenös tomografi",
#     17800: "Spinal angiografi",
    "22200": "Arbetsprov",
    "22203": "Arbetsprov armcykling",
    "22204": "Arbetsprov på rullande matta",
    "22205": "Arbetsprov med pulsoximetri",
    "22206": "Arbetsprov med PEF",
    "22208": "Arbetsprov med pulsoximetri och PEF",
    
    "23210": "Arbetsprov med blodgaser",
    
    "32000": "Lungor",
    "32028": "Lungor endast genomlysning",
    "32030": "Lungor tomosyntes",
    "32300": "Lungor undersökning i liggande",
    "32320": "Lungor, liggande barn (frontal- och sidobild)",
    "37300": "Coronarangiografi ett eller flera kärl",
    "39600": "Coronarangiografi med PCI utan stent",
    "39648": "Coronarangiografi med PCI med stent",
  
    "72110": "Scintigrafi myokard vila och belastning",
    "72120": "Scintigrafi myokard vila och arbete",
    
    "73110": "Scintigrafi lungor",
    "73130": "Scintigrafi lungor ventilation perfusion med fördelning",
    "73400": "Scintigrafi tomografi lungor",
    "73403": "Scintigrafi tomografi inklusive DT lungor",

    "83000": "DT thorax utan iv kontrast",
    "83007": "DT thorax översiktsundersökning",
    "83050": "DT thorax med punktion/biopsi",
    "83051": "DT thorax med punktion och kateterinläggning",
    "83076": "DT thorax högupplösande (HRCT)",
    "83077": "DT thorax med låg stråldos",
    "83080": "DT thorax med iv kontrast",
    "83081": "DT thorax utan och med iv kontrast",
    
    "83100": "DT hjärta",
    "83180": "DT hjärta med iv kontrast",
    "83600": "DT thorax och övre buk utan iv kontrast",
    "83680": "DT thorax och övre buk med iv kontrast",
    "83681": "DT thorax och övre buk utan och med iv kontrast",
    "83900": "DT thorax och buk utan iv kontrast",
    "83980": "DT thorax och buk med iv kontrast",
    "83981": "DT thorax och buk utan och med iv kontrast",
    
    "84000": "DT buk utan iv kontrast",
    "84050": "DT buk med punktion",
    "84051": "DT buk med dräninläggning",
    "84080": "DT buk med iv kontrast",
    "84081": "DT buk utan och med iv kontrast",

    "92000": "Ekokardiografi",
    "92007": "Ekokardiografi riktad undersökning",
    "92072": "Ekokardiografi 3D",
    "92200": "Ekokardiografi transesofagalt",
    
    "94000": "ULJ buk",
    "94050": "ULJ buk med punktion/biopsi",
    "94051": "ULJ buk med kateterinläggning",
    "94081": "ULJ buk utan och med iv kontrast",
    
    "M3100": "MRT hjärta",
    "M3107": "MRT hjärta kort undersökning",
    "M3127": "MRT hjärta funktionsstudie",
    "M3181": "MRT hjärta utan och med iv kontrast"
}

In [ ]:
for k in sorted(sectra_b["Undersökningskod"].value_counts().keys()):
    #if k not in sectra_koder.keys():
        print(k)

In [ ]:
len(sectra)

In [ ]:
len(sectra_new_df)

In [ ]:
list(sectra_new_df)

In [ ]:
sectra_new_df.iloc[835:845]

In [ ]:
sectra_new_df[sectra_new_df["Undersökningskod"].isna()].head()

In [ ]:
print(len(sectra))
print(len(sectra_new_df))

In [ ]:
vc = sectra["Undersökningskod"].value_counts()
vc_new = sectra_new_df["Undersökningskod"].value_counts()

for k in sorted(set(list(vc.keys()) + list(vc_new.keys()))):
    print(f"{k}\t{vc[k]}\t{vc_new[k]}\t{vc_new[k]-vc[k]}")

In [ ]:
def print_sectra_code_counts(df):
    counts = df["Undersökningskod"].value_counts()
    counts_recent = df[pd.to_datetime(df["Undersökningsdatum"]) >= pd.to_datetime("2017-01-01")]["Undersökningskod"].value_counts()
    for k in sorted(counts.keys()):
        n_recent = counts_recent[k] if k in counts_recent else 0
        print(f'{k}~{sectra_koder[k]}~{counts[k]}~{n_recent}')
        
print_sectra_code_counts(sectra_new_df)

In [ ]:
sectra["Undersökningsdatum"].sort_values()

In [ ]:
sectra["Fullständigt svar"].value_counts().head(10)

In [ ]:
sectra.dtypes

### Ny Sectra data

In [ ]:
sectra_new_df = pd.read_csv("/mnt/air-crypt/air-crypt-raw/andersb/data/Skane-1718-SECTRA-2nd/20210924/SECTRA_report_data_2012_2018_20210831.utf8.massaged.csv", sep="|", encoding="UTF8", encoding_errors="strict")
sectra_new_df.head()

In [ ]:
sectra_new_df.dtypes

In [ ]:
print(len(sectra))
print(len(sectra_new_df))

In [ ]:
sectra_new_df["Undersökningskod"].value_counts()

## Jakob/Donika -- yrsel

In [461]:
len(set(liggaren["Alias"]))

325539

In [284]:
importlib.reload(s1718_util)
import mim.extractors.skane_1718_extract_util as s1718_util

BLOOD_SAMPLES = [
    "P-Kalium",
    "P-Kreatinin (enz)",
    "B-Hemoglobin (Hb)",
    "P-Natrium",
    "P-Glukos",
    "P-CRP",
    "B-Calciumjon",
    "P-Laktat",
    "P-Troponin T",
    "B-Trombocyter",
    "S-Calciumjon (pH7.4)",
    "aB-Laktat",
    "P-Calcium",
    "P-NT-proBNP",
    "B-CRP (PNA)",
    "B-Hb (PNA)"
]
BLOOD_SAMPLE_MAX_HOURS_AFTER_INDEX=12


ATC_PREFIXES = [
    "C03", "C07", "C08", "C09",
    "N02A", "N02", "N05", "N06", 
    "M01", "N03"
]
ATC_DAYS_BEFORE_INDEX = [180, 365*5]


TARGET_OUTCOME_ICD_PREFIXES = ["H80-H83"]
TARGET_OUTCOME_ICD_PREFIXES += ["I60", "I61", "I62", "I63", "I64", "I65", "I66", "I67", "I68", "I69"]
TARGET_OUTCOME_ICD_PREFIXES += ["I440", "I441A", "I441B", "I442", "I443", "I456", "I456A", "I460", "I461", "I469", "I47", "I470", "I472", "I472C", "I48", "I490", "I495", "I495A", "I495B", "I495C", "I498D"]
TARGET_OUTCOME_ICD_PREFIXES = list(sorted(set(TARGET_OUTCOME_ICD_PREFIXES)))

TARGET_OUTCOME_KVA_CODES = ["DF013", "DF025", "DF030",
                            "FPE00", "FPE10", "FPE20", "FPE26", "FPE96",
                            "FPF00", "FPF10", "FPF20", "FPF96",
                            "FPG10", "FPG20", "FPG30", "FPG33", "FPG36", "FPG96",
                            "TFP00"]

TARGET_OUTCOME_KVA_CODES += ["AA011", "AA012", "AA013", "AA014", "AA015", "AA016", "AA017", "AA021", "AA022", "AA023", "AA024", "AA045", "AA046", "AA047", "AA048", "AA049", "AA050", "AA051", "AF012", "AF039", "AF040", "AF041", "AF042", "AF043", "AF054", "AF055", "AF056", "AF057", "AF058", "AF059", "AF060", "AF061", "AF062", "AF063", "AF074", "AF075", "FPE00", "FPE10", "FPE20", "FPE26", "FPE30", "FPE40", "FPE96", "FPF00", "FPF20", "FPF40", "FPF96", "FPG30", "FPG33", "FPG36", "FPG40", "FPG43", "FPG96", "FPJ00", "FPK00"]
TARGET_OUTCOME_KVA_CODES = list(sorted(set(TARGET_OUTCOME_KVA_CODES)))


TARGET_MELIOR_DISCHARGE_DIAGS = ["H810", "H811", "H812", "H813", "H814", "H818", "H819"]
TARGET_MELIOR_DISCHARGE_DIAGS += ["R42","R559","I48","I60","I61","I62","I63","I64","I65","I66","I67","I68","I69"]
TARGET_MELIOR_DISCHARGE_DIAGS = list(sorted(set(TARGET_MELIOR_DISCHARGE_DIAGS)))

OUTCOME_DAYS_AFTER_INDEX = [30, 90, 365]


PREV_DISEASE_ICD_PREFIXES = [f"E1{i}" for i in range(0, 6)] + ["H80-H83"] + \
 [f"I1{i}" for i in range(0, 6)] + [f"I2{i}" for i in range(0, 6)] + \
 [f"I6{i}" for i in range(0, 10)] + [f"I7{i}" for i in range(0, 10)]

TARGET_OUTCOME_REVISIT_CAUSES = ["Yrsel"]

MELIOR_30DAY_ICD_PREFIXES = [
    "S02", "S12", "S22", "S32", "S42", "S52", "S62", "S72", "S82", "S92",
    "W01", "W18", "W19"
]


TARGET_MELIOR_DISCHARGE_DIAGS = ["H810", "H811", "H812", "H813", "H814", "H818", "H819",
                                 "I48", 
                                 "I60", "I61", "I62", "I63", "I64", "I65", "I66", "I67", "I68", "I69",
                                 "R42", "R559"]


SOS_MELIOR_EXPANSION_DICT = {
    "prev5y-H80-H83": ["H811", "H813", "H814"]
}


YRSEL_EXPORT_ROOT = "/mnt/air-crypt/air-crypt-esc-trop/andersb/temp-skane-1718/yrsel-export"
LIGGARE_KEEP_COLUMNS=[
    "KontaktId", "Vardkontakt_InskrivningDatum", "BesokOrsak_Kod", "Ambulans_Arrival_Liggare_AB",
    "Patient_Kon", "Vardkontakt_PatientAlderVidInskrivning", 
    "Inlagd", "Utskriven till", "Vårddtid på akuten i minuter", "Tid till läkare på akuten i minuter", "Triagekategori",
    
]

OUT_DROP_COLUMNS = ["Vardkontakt_InskrivningDatum", "VårdtillfälleFörDiagnos_StartDatum", "VårdtillfälleFörDiagnos_SlutDatum", "Date_of_stroke"]



t0 = time.time()

CUT = 1000
CUT = -1000
CUT = 0

yrsel_liggare = liggaren[liggaren["BesokOrsak_Kod"] == "Yrsel"].copy()
print(f"Totalt antal besök: {len(yrsel_liggare)}")
yrsel_liggare = yrsel_liggare.sort_values("Vardkontakt_InskrivningDatum").drop_duplicates("Alias")
print(f"Endast första besöket: {len(yrsel_liggare)}")
yrsel_liggare = yrsel_liggare.join(triage_df.set_index("KontaktId"),"KontaktId")
yrsel_liggare["Triagekategori"] = yrsel_liggare["Triagekategori"].fillna("--SAKNAS--")

yrsel_liggare.loc[:,"Vardkontakt_InskrivningDatum"] = pd.to_datetime(yrsel_liggare["Vardkontakt_InskrivningDatum"]) # Cast
print(len(yrsel_liggare))
yrsel_liggare = yrsel_liggare.set_index("Alias").join(sos_nyckel.set_index("Alias")) # join SoS key
yrsel_liggare = yrsel_liggare[yrsel_liggare["LopNr"].notna()]  # Drop 333 where there is no SoS entry.
yrsel_liggare.loc[:,"LopNr"] = yrsel_liggare["LopNr"].astype(int)  # Cast
print(len(yrsel_liggare))
if CUT != 0:
    #yrsel_liggare = yrsel_liggare.iloc[:CUT]
    if CUT > 0:
        yrsel_liggare = yrsel_liggare.head(CUT)
    else:
        yrsel_liggare = yrsel_liggare.tail(-CUT)
print(len(yrsel_liggare))



prev_diseases = s1718_util.extract_previous_diagnoses_combine_sv_ov(sos_oppen, sos_sluten, yrsel_liggare.reset_index(), PREV_DISEASE_ICD_PREFIXES)
outcome_icds_sos = s1718_util.extract_icd_outcomes_combine_sv_ov(sos_oppen, sos_sluten, yrsel_liggare.reset_index(), TARGET_OUTCOME_ICD_PREFIXES, OUTCOME_DAYS_AFTER_INDEX)
outcome_kva_sos = s1718_util.extract_kva_outcomes_combine_sv_ov(sos_oppen, sos_sluten, yrsel_liggare.reset_index(), TARGET_OUTCOME_KVA_CODES, OUTCOME_DAYS_AFTER_INDEX)
lm = s1718_util.extract_historical_atc_sos(sos_lakemedel, yrsel_liggare.reset_index(),
                                           ATC_DAYS_BEFORE_INDEX, ATC_PREFIXES)
lm_compl = s1718_util.extract_historical_atc_sos(sos_lakemedel, yrsel_liggare.reset_index(),
                                           ATC_DAYS_BEFORE_INDEX[1], ATC_PREFIXES, ATC_DAYS_BEFORE_INDEX[0])

lb = s1718_util.extract_index_blood_samples(labb, yrsel_liggare.reset_index(),
                                            BLOOD_SAMPLES, BLOOD_SAMPLE_MAX_HOURS_AFTER_INDEX)
death = s1718_util.extract_death_sos_dors(sos_dors, yrsel_liggare.reset_index(), 
                                         OUTCOME_DAYS_AFTER_INDEX)
outcome_revisit = s1718_util.extract_liggare_future_visits(liggaren, liggaren_followup, yrsel_liggare.reset_index(), TARGET_OUTCOME_REVISIT_CAUSES, OUTCOME_DAYS_AFTER_INDEX)

melior_discharge_diags_df = s1718_util.extract_melior_discharge_diags(melior_diagnoser_vid_kontakt, yrsel_liggare.reset_index(), TARGET_MELIOR_DISCHARGE_DIAGS)
index_start_stop_df = s1718_util.get_index_start_stop(melior_diagnoser_vid_kontakt, yrsel_liggare.reset_index())
melior_outcome_df = s1718_util.contact_and_30day_melior_outcomes(melior_diagnoser_vid_kontakt, melior_diagnoser_30dagar_efter, yrsel_liggare.reset_index(), MELIOR_30DAY_ICD_PREFIXES)
charlson_df = s1718_util.compute_charlson_melior_5y(melior_diagnoser_5ar_fore, yrsel_liggare.reset_index())


#out = yrsel_liggare.copy()
out = yrsel_liggare[LIGGARE_KEEP_COLUMNS].copy()
out = out.join([index_start_stop_df, death, lb, prev_diseases, charlson_df])
expanded_sos_melior_history_df = s1718_util.expand_grouped_sos_icd_by_melior_historic(melior_diagnoser_5ar_fore, out.reset_index(), SOS_MELIOR_EXPANSION_DICT)
out = out.join(expanded_sos_melior_history_df)
out = out.join([lm, lm_compl])
new_lm = s1718_util.infer_newly_introduced_meds(out, ATC_PREFIXES, ATC_DAYS_BEFORE_INDEX[0], ATC_DAYS_BEFORE_INDEX[1])
out = out.join([new_lm, melior_outcome_df])
s_w_series = s1718_util.combine_S_W_melior(out, MELIOR_30DAY_ICD_PREFIXES)
out = out.join(s_w_series)
out = out.join([outcome_revisit, melior_discharge_diags_df, outcome_kva_sos, outcome_icds_sos])
stroke_date_df = s1718_util.find_first_stroke(sos_oppen, sos_sluten, out)
stroke_outcome_combined_series = [s1718_util.combine_cols_any(out, [f"outcome-{days}d-I{i}" for i in range(60,70)], f"outcome-{days}d-any-I60-I69") for days in OUTCOME_DAYS_AFTER_INDEX]
out = out.join([stroke_date_df])
out = out.join(stroke_outcome_combined_series)
FNAME = "yrsel-2022-04-21.csv"
FNAME_DROPPED = FNAME[:-4] + ".dropped_dates.csv"
out.to_csv(FNAME)
out.drop(OUT_DROP_COLUMNS, axis=1).to_csv(FNAME_DROPPED)

t1 = time.time()
print(f"Time: {t1-t0=}")
print(f"Shape: {out.shape}")

Totalt antal besök: 15773
Endast första besöket: 13763
13763
13624
13624


13624it [00:21, 647.02it/s]
13624it [00:15, 870.48it/s]
13624it [00:33, 402.35it/s]
13624it [00:38, 354.11it/s]
13624it [00:28, 481.98it/s]
13624it [00:35, 383.99it/s]
13624it [00:50, 272.01it/s]
13624it [00:54, 248.98it/s]
13624it [00:54, 248.74it/s]
13624it [01:40, 135.79it/s]
13624it [00:03, 3609.03it/s]
13624it [00:24, 554.80it/s]
13624it [01:35, 143.32it/s]
13624it [00:34, 398.05it/s]
13624it [00:48, 282.23it/s]
13624it [00:08, 1634.96it/s]
13624it [00:13, 987.83it/s] 


Time: t1-t0=675.6900773048401
Shape: (13624, 463)


In [287]:
importlib.reload(s1718_util)
import mim.extractors.skane_1718_extract_util as s1718_util

charlson_df = s1718_util.compute_charlson_melior_5y(melior_diagnoser_5ar_fore, yrsel_liggare.reset_index())

13624it [00:47, 287.67it/s]


In [288]:
importlib.reload(s1718_util)
import mim.extractors.skane_1718_extract_util as s1718_util

charlson_df = s1718_util.compute_charlson_melior_5y(melior_diagnoser_5ar_fore, yrsel_liggare.reset_index())

13624it [00:48, 281.76it/s]


In [305]:
out.filter(regex="med-1825d.*").sum()

med-1825d-C03     1396
med-1825d-C07     1654
med-1825d-C08     1233
med-1825d-C09     1919
med-1825d-N02A    2050
med-1825d-N02     3028
med-1825d-N05     2100
med-1825d-N06     1483
med-1825d-M01     2129
med-1825d-N03      456
dtype: int64

In [313]:
out["X"] = out.apply(lambda row: 1 if not row["med-180d-C03"] and row["med-1825d-C03"] else 0, axis=1)

In [315]:
out["X"].sum()

528

In [318]:
m = out.filter(regex="med-180d.*").values + out.filter(regex="med-1825d.*").values
print(sum(m))
m[m==2] = 0
print(sum(m))
m

[2264 2967 2072 3495 2720 4570 3348 2422 2670  711]
[ 528  341  394  343 1380 1486  852  544 1588  201]


array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [297]:
m = out.filter(regex="med-180d.*").values * 10 + out.filter(regex="med-1825d.*").values
print(m.sum())
m[m==11] = 0
print(m.sum())
m[m==1] = 0
print(m.sum())

m

115358
7657
0


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [280]:
icd_outcomes_ov

,outcome-30d-H80-H83-OV,outcome-30d-I440-OV,outcome-30d-I441A-OV,outcome-30d-I441B-OV,outcome-30d-I442-OV,outcome-30d-I443-OV,outcome-30d-I456-OV,outcome-30d-I456A-OV,outcome-30d-I460-OV,outcome-30d-I461-OV,...,outcome-365d-I60-OV,outcome-365d-I61-OV,outcome-365d-I62-OV,outcome-365d-I63-OV,outcome-365d-I64-OV,outcome-365d-I65-OV,outcome-365d-I66-OV,outcome-365d-I67-OV,outcome-365d-I68-OV,outcome-365d-I69-OV
Alias,,,,,,,,,,,,,,,,,,,,,
10225654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10170762,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10165859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10284405,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10055648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10048349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10112154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10060530,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [251]:
## TODO


# Generation
def first_stroke():
    pass

def scb_data():
    pass

def compute_charlson():
    pass

def find_kva_within_index_visit():
    pass


# Post-processing:

def join_sos_sv_ov_icd(df):
    pass

def join_sos_sv_ov_kva(df):
    pass

def infer_new_medication(df):
    pass

def compute_index_hospital_stay_length(df):
    pass

def combine_w_s_diagnoses_melior(df):
    pass

out

,KontaktId,AktuellSjukhusId,Sjukhus_Namn,BesokOrsakId,BesokOrsak_Kod,BesokOrsak_Beskrivning,Vardkontakt_InskrivningDatum,Vardkontakt_UtskrivningDatum,Vardkontakt_PatientAlderVidInskrivning,Patient_Kon,...,outcome-365d-I60-OV,outcome-365d-I61-OV,outcome-365d-I62-OV,outcome-365d-I63-OV,outcome-365d-I64-OV,outcome-365d-I65-OV,outcome-365d-I66-OV,outcome-365d-I67-OV,outcome-365d-I68-OV,outcome-365d-I69-OV
Alias,,,,,,,,,,,,,,,,,,,,,
10225654,K1000262,O10001,SUS Lund,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-01-01 10:58:00,2017-01-01 19:01:00,56,F,...,0,0,0,0,0,0,0,0,0,0
10170762,K1000350,SE162321000255-O16571,Ystads lasarett,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-01-01 12:50:00,2017-01-01 18:34:00,79,M,...,0,0,0,0,0,0,0,0,0,0
10165859,K1000445,SE162321000255-O11016,Lasarettet Trelleborg,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-01-01 14:36:00,2017-01-01 17:27:00,87,F,...,0,0,0,0,0,0,0,0,0,0
10284405,K1000450,O10001,SUS Lund,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-01-01 14:44:00,2017-01-02 02:24:00,80,F,...,0,0,0,0,0,0,0,0,0,0
10055648,K1000478,SE162321000255-O12014,Helsingborgs lasarett,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2017-01-01 15:07:00,2017-01-01 15:46:00,43,M,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10313317,K1632519,SE162321000255-O10532,Centralsjukhuset Kristianstad,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2018-12-30 16:51:00,2018-12-30 18:54:00,76,M,...,0,0,0,1,0,0,0,0,0,0
10119414,K1632525,O11249,Hässleholms sjukhus,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2018-12-30 16:55:00,2018-12-30 19:03:00,91,M,...,0,0,0,0,0,0,0,0,0,0
10306701,K1632599,SE162321000255-O12014,Helsingborgs lasarett,382.0,Yrsel,"Yrsel och svindel RETTS-A 11, Yrsel och balans...",2018-12-30 18:35:00,2018-12-30 23:08:00,61,M,...,0,0,0,0,0,0,0,0,0,0


In [157]:
###
### TODO
###
### Kolla melior_diagnoser_vid_kontakt. 
### - Finns där en diagnos för varje kontakt (som inte avvikit/avlidit)?
### - Finns där epikris för alla som lagts in?
### - Finns där något systematiskt för de som skickats hem (som endast behandlas i öppenvården)?
### 
### -> Kan man då på ett precist vis definiera slutdiagnoser vid ett besök på akuten?
### <- Verkar som det finns kontakter som inte finns med i den melior-datan. Hur många är de?

In [ ]:
def associate_sos_entry_liggare(liggar_df, sos_sv, sos_ov):
    sv_df = sos_sv[sos_sv["Alias"].isin(liggar_df["Alias"])]
    ov_df = sos_ov[sos_ov["Alias"].isin(liggar_df["Alias"])]
    for _, (alias, index_date, till, inlagd) in tqdm(
            liggar_df[["Alias", "Vardkontakt_InskrivningDatum", "Utskriven till", "Inlagd"]].iterrows()
    ):
        sv = sv_df[(sv_df["Alias"] == alias) & (index_date + timedelta(days=-1) <= sv_df["INDATUM"]) & (sv_df["INDATUM"] <= index_date + timedelta(days=1))]
        print(f"{alias}\t{inlagd=}\t{len(sv)=}")
        

cols = ["Alias", "SV_Discharge_Date", "SV_Discharge_HDIA", "OV_Date", "OV_Discharge_HDIA"]

associate_sos_entry_liggare(yrsel_liggare.reset_index(),sos_sluten,sos_oppen)

In [ ]:
list(sos_sluten)

In [ ]:
(liggaren["Inlagd"].astype(str) + "  ~  " +liggaren["Utskriven till"]).value_counts()

In [ ]:
sos_sluten[sos_sluten["Alias"] == 10170762]

In [ ]:
yrsel_liggare[yrsel_liggare["Inlagd"] == 1]

In [ ]:
sos_sluten["OP"]

## Jakob/Daniel -- Geriatri

In [ ]:
geriatri_liggare = liggaren.sort_values("Vardkontakt_InskrivningDatum").drop_duplicates("Alias")
print(f"Antal unika patienter: {len(geriatri_liggare)}")
geriatri_liggare = geriatri_liggare[geriatri_liggare["Vardkontakt_PatientAlderVidInskrivning"] >=75]
print(f"Antal unika patienter över 75: {len(geriatri_liggare)}")

In [ ]:
a = liggaren[liggaren["Vardkontakt_PatientAlderVidInskrivning"] >= 75]["Alias"]
print(f"Antal besök av patienter över 75: {len(a)}")
print(f"Antal patienter med fler än 3 besök: {(a.value_counts() > 3).sum()}")

## Ulf -- Descriptive

In [187]:
list(liggaren)

['KontaktId',
 'Alias',
 'AktuellSjukhusId',
 'Sjukhus_Namn',
 'BesokOrsakId',
 'BesokOrsak_Kod',
 'BesokOrsak_Beskrivning',
 'Vardkontakt_InskrivningDatum',
 'Vardkontakt_UtskrivningDatum',
 'Vardkontakt_PatientAlderVidInskrivning',
 'Patient_Kon',
 'Utskriven till',
 'Inläggningsavdelning',
 'Vårddtid på akuten i minuter',
 'Tid till läkare på akuten i minuter',
 'Första läkarkontakten på akuten',
 'Avvikit mot rekommendation',
 'Inlagd',
 'Process_text',
 'UppföljningParameter_text']

In [189]:
liggaren["Process_text"].value_counts()

-                 176850
Ankomst           161345
Nr.lapp-reg        79288
Triage             37950
VR                 26482
                   ...  
VPL AKM Team 2         1
Närsjv/OBS             1
Barntraumalarm         1
CT CSK                 1
Team 5                 1
Name: Process_text, Length: 165, dtype: int64

In [199]:
svar["AnkomstsättAkutvårdsregistrering"].value_counts()

Egen inställelse (Gående, egen bil, taxi och dyl.)    500332
Ambulans                                              152191
Name: AnkomstsättAkutvårdsregistrering, dtype: int64

## Axel BRSM-AMI

In [45]:
list(liggaren)

['KontaktId',
 'Alias',
 'AktuellSjukhusId',
 'Sjukhus_Namn',
 'BesokOrsakId',
 'BesokOrsak_Kod',
 'BesokOrsak_Beskrivning',
 'Vardkontakt_InskrivningDatum',
 'Vardkontakt_UtskrivningDatum',
 'Vardkontakt_PatientAlderVidInskrivning',
 'Patient_Kon',
 'Utskriven till',
 'Inläggningsavdelning',
 'Vårddtid på akuten i minuter',
 'Tid till läkare på akuten i minuter',
 'Första läkarkontakten på akuten',
 'Avvikit mot rekommendation',
 'Inlagd',
 'Process_text',
 'UppföljningParameter_text']

In [50]:
liggaren[liggaren["KontaktId"] == "K1596729"]

,KontaktId,Alias,AktuellSjukhusId,Sjukhus_Namn,BesokOrsakId,BesokOrsak_Kod,BesokOrsak_Beskrivning,Vardkontakt_InskrivningDatum,Vardkontakt_UtskrivningDatum,Vardkontakt_PatientAlderVidInskrivning,Patient_Kon,Utskriven till,Inläggningsavdelning,Vårddtid på akuten i minuter,Tid till läkare på akuten i minuter,Första läkarkontakten på akuten,Avvikit mot rekommendation,Inlagd,Process_text,UppföljningParameter_text
602607,K1596729,10233388,SE162321000255-O11414,SUS Malmö,120.0,BröstSm,"Bröstsmärta/Bröstkorgssmärta UNS RETTS-A 5, RE...",2018-11-18 04:46:00,2018-11-18 11:28:00,80,M,Avdelning,Intensivv avd,402.0,87.0,2018-11-18 06:13:00,0,1,Triage,Ambulans


In [56]:
LIGGARE_KEEP_COLUMNS=[
    "Vardkontakt_InskrivningDatum", "Sjukhus_Namn", "BesokOrsak_Kod",
    "Patient_Kon", "Vardkontakt_PatientAlderVidInskrivning",
    "Inlagd", "Utskriven till", "Vårddtid på akuten i minuter", "Tid till läkare på akuten i minuter", "Triagekategori"
]

PREV_ICDS = list(sorted(set(["A30-A49","B96","C34","C43-C44","C50","C60-C63","C64-C68","C78","C90","D64","E03","E119","E78","E87","F10","F11","F19","F31","F32","F33","F41","F43","F60","F90","G40","G45","H00-H06","H15-H22","H25-H28","H30-H36","H40-H42","H43-H45","H60-H62","H65-H75","H90-H95","I109","I209","I214","I251","I252","I259","I350","I480","I482","I489","I509","I639","I693","J159","J189","J441","J449","J459","K922","L00-L08","L20-L30","L40-L45","L55-L59","L60-L75","L80-L99","M05","M16","M17","M17","M48","M48","M54","M75","M79","N10","N18","N18","N20","N20","N39","N39","N40-N51","N40-N51","N80-N98","N80-N98","O20-O29","O30-O48","O60-O75","O80-O84"])))
TARGET_OUTCOME_ICD_PREFIXES = ["I200", "I21", "I22"]
OUTCOME_DAYS_AFTER_INDEX = [30]

ATC_DAYS_BEFORE_INDEX = [180, 5*365]
ATC_PREFIXES = ["A01A","A02BC","A06AB","A06AC","A06AD","A10AB","A10AC","A10AD","A10AE","A10BA","A11C","A11E","A12A","A12B","B01AA03","B01AC04","B01AC06","B01AF01","B01AF02","B03AA07","B03BA01","B03BB01","C01AA05","C01DA02","C01DA14","C03AA01","C03CA01","C03DA01","C03EA01","C07AB02","C07AB03","C07AB07","C08CA01","C08CA02","C09AA02","C09AA05","C09CA01","C09CA06","C09DA01","C10AA01","C10AA05","D01A","D02A","D07A","G03A","G03C","G04B","G04C","H02AB","H03AA","J01AA","J01CA","J01CE","J01CF","J01MA","J01XE","L02B","L04A","M01AB","M01AE","M04AA","M05BA","N02AA","N02AJ","N02AX","N02BE","N02CC","N03AF","N03AG","N03AX","N04BA","N04BC","N05AH","N05AX","N05BA","N05BB","N05CD","N05CF","N05CM","N06AA","N06AB","N06AX","N06BA","N06DA","N06DX","R01A","R03AC","R03AK","R03BA","R03BB","R05CB","R05FA","R06AD","R06AE","R06AX","S01A","S01B","S01E","S01X","S03C"]

BLOOD_SAMPLES = [
    "P-Troponin T",
    "P-Kreatinin (enz)",
    "P-Glukos",
    "B-Hemoglobin (Hb)",
]
BLOOD_SAMPLE_MAX_HOURS_AFTER_INDEX=12


CUT = 500
CUT = -1

t0 = time.time()

brsm_liggare = liggaren[liggaren["BesokOrsak_Kod"] == "BröstSm"].copy()
print(f"Totalt antal besök: {len(brsm_liggare)}")
brsm_liggare = brsm_liggare.join(triage_df.set_index("KontaktId"),"KontaktId")
brsm_liggare["Triagekategori"] = brsm_liggare["Triagekategori"].fillna("--SAKNAS--")
brsm_liggare.loc[:,"Vardkontakt_InskrivningDatum"] = pd.to_datetime(brsm_liggare["Vardkontakt_InskrivningDatum"]) # Cast
print(len(brsm_liggare))
brsm_liggare = brsm_liggare.set_index("Alias").join(sos_nyckel.set_index("Alias")) # join SoS key
brsm_liggare = brsm_liggare[brsm_liggare["LopNr"].notna()]  # Drop where there is no SoS entry.
brsm_liggare.loc[:,"LopNr"] = brsm_liggare["LopNr"].astype(int)  # Cast
print(len(brsm_liggare))
if CUT > 0:
    brsm_liggare = brsm_liggare.iloc[:CUT]
print(len(brsm_liggare))

prev_diseases_ov = s1718_util.extract_previous_diagnoses_sos_ov(sos_oppen, brsm_liggare.reset_index(),PREV_ICDS)
prev_diseases_sv = s1718_util.extract_previous_diagnoses_sos_sv(sos_sluten, brsm_liggare.reset_index(),PREV_ICDS)


icd_outcomes_sv = s1718_util.extract_outcome_icds_sos(sos_sluten, brsm_liggare.reset_index(),
                                                  TARGET_OUTCOME_ICD_PREFIXES,
                                                  OUTCOME_DAYS_AFTER_INDEX,
                                                     "SV")
icd_outcomes_ov = s1718_util.extract_outcome_icds_sos(sos_oppen, brsm_liggare.reset_index(),
                                                  TARGET_OUTCOME_ICD_PREFIXES,
                                                  OUTCOME_DAYS_AFTER_INDEX,
                                                     "OV")
death = s1718_util.extract_death_sos_dors(sos_dors, brsm_liggare.reset_index(), 
                                         OUTCOME_DAYS_AFTER_INDEX)
lm = s1718_util.extract_historical_atc_sos(sos_lakemedel, brsm_liggare.reset_index(),
                                           ATC_DAYS_BEFORE_INDEX, ATC_PREFIXES)
lb = s1718_util.extract_index_blood_samples(labb, brsm_liggare.reset_index(),
                                            BLOOD_SAMPLES, BLOOD_SAMPLE_MAX_HOURS_AFTER_INDEX)


out = pd.concat([brsm_liggare, prev_diseases_sv, prev_diseases_ov, lm, icd_outcomes_sv, icd_outcomes_ov, death, lb], axis=1)

out.to_csv("brsm-U.csv")

t1 = time.time()
print(f"Time: {t1-t0=}")

Totalt antal besök: 51402
51402
50565
50565


50565it [01:33, 542.89it/s]
50565it [00:59, 852.13it/s]
50565it [01:04, 785.58it/s]
50565it [01:30, 558.96it/s]
50565it [00:12, 3920.11it/s]
50565it [13:15, 63.53it/s]
50565it [15:35, 54.05it/s]
50565it [04:23, 191.83it/s]


Time: t1-t0=2324.519427537918


## MelanderDiabetes

In [ ]:
# Urval:
# 1. Sökt för hyperglykemi (BlskrHög)
# 2. Diabetesläkemedel inom 12m före besök (ATC kod börjar på A10)
# 3. Diagnoskoder i det förflutna som indikerar diabetes (E10, E11, E12, E13, E14)
# 4. P-glukos >= 11 i samband med besök

In [236]:
# 1.
hyperglykemi_liggare = liggaren[liggaren["BesokOrsak_Kod"] == "BlskrHög"]

In [237]:
# 2.
diabetes_lakemedel = sos_lakemedel[sos_lakemedel["ATC"].str.startswith("A10")]

In [238]:
# 3.
diabetes_sluten = sos_sluten[~sos_sluten['hdia'].isna()]
diabetes_sluten = diabetes_sluten[diabetes_sluten['hdia'].str.startswith("E1")]

In [239]:
# 4.
diabetes_labb = labb[(labb["Labanalys_Namn"] == "P-Glukos")]
diabetes_labb = diabetes_labb[(pd.to_numeric(diabetes_labb["Analyssvar_Varde"], errors='coerce') >= 11)]

In [250]:
DANEHAMMAR_BESOKSORSAKER_LONG_LIST = ["Anemi RETTS-A 48,RETTS-P 148"]
DANEHAMMAR_BLOOD_SAMPLES = []


danehammar_liggare = liggaren[liggaren["BesokOrsak_Beskrivning"].isin(DANEHAMMAR_BESOKSORSAKER_LONG_LIST)]


In [240]:
a_sok = set(hyperglykemi_liggare["Alias"])
a_lm = set(diabetes_lakemedel["Alias"])
a_sv = set(diabetes_sluten["Alias"])
a_labb = set(diabetes_labb["Alias"])
a_non_bs_union = a_sok | a_lm | a_sv
a_union = a_sok | a_lm | a_sv | a_labb

In [241]:
print(len(a_sok))
print(len(a_lm))
print(len(a_sv))
print(len(a_labb))
print()
print(len(a_non_bs_union))
print(len(a_union))

1356
33697
3544
18970

33955
39795


In [ ]:
print(len(set(liggaren["Alias"])))
print(len(liggaren))

In [ ]:
liggaren["Alias"].isin(a_union).sum()

## Åtgärder

In [ ]:
atgarder_30_dagar = pd.read_csv(FILES["MELIOR_Åtgärder30DagarEfterVårdkontakt_2017_2018.csv"], sep="|", encoding="ISO-8859-1")
atgarder_vid_kontakt = pd.read_csv(FILES["MELIOR_ÅtgärderVidVårdkontakt_2017_2018.csv"], sep="|", encoding="ISO-8859-1")
atgarder_till_2020 = pd.read_csv(FILES["MELIOR_PatientAtgarder_Efter_Vardkontakt_UtskrivningDatum_Till_20191231.csv"], sep="|", encoding="ISO-8859-1")


In [ ]:
a = list(atgarder_30_dagar)
b = list(atgarder_vid_kontakt)
c = list(atgarder_till_2020)
a

In [ ]:
b

In [ ]:
c

In [ ]:
x = atgarder_30_dagar["PatientAtgard_Kod"].value_counts()

In [ ]:
atgarder_30_dagar.head()

## Axel Check

In [ ]:
pat_165 = pd.read_csv("/home/sapfo/andersb/165_pat.txt",sep="\t")
pat_165

In [ ]:
i200_pats = pd.read_csv("/home/sapfo/andersb/i200_only_melior.csv")
i200_pats

In [ ]:
distant_future = datetime(year=2025,month=12,day=31)

diag_prefix="I200"  # "I21"
df = i200_pats # "pat_165"


def lookup(alias, diag_date):
    a = dfs["melior_pre_5yr"][dfs["melior_pre_5yr"]["Alias"] == alias].drop(["TermId", "KontaktId"], axis=1).fillna({"VårdtillfälleFörDiagnos_SlutDatum": "n/a"})
    b = dfs["melior_kontakt"][dfs["melior_kontakt"]["Alias"] == alias].drop("KontaktId", axis=1).fillna({"VårdtillfälleFörDiagnos_SlutDatum": "n/a"})
    c = dfs["melior_post_30dagar"][dfs["melior_post_30dagar"]["Alias"] == alias].drop(["TermId", "KontaktId"], axis=1).fillna({"VårdtillfälleFörDiagnos_SlutDatum": "n/a"})

    for df in [a, b, c]:
        df["VårdtillfälleFörDiagnos_SlutDatum"] = df["VårdtillfälleFörDiagnos_SlutDatum"].replace("n/a", str(distant_future))
        df['vtkey'] = df["VårdtillfälleFörDiagnos_StartDatum"] + "~" + df["VårdtillfälleFörDiagnos_SlutDatum"]

    vtkeys = set(pd.concat([a['vtkey'], b['vtkey'], c['vtkey']]))
    m_table = {}

    for vtkey in vtkeys:
        v = ""
        if (a['vtkey'] == vtkey).sum() > 0:
            v += "Pre-5-years<br/>"
        if (b['vtkey'] == vtkey).sum() > 0:
            v += "Contact<br/>"
        if (c['vtkey'] == vtkey).sum() > 0:
            v += "Post 30d<br/>"
        m_table[vtkey] = v

    cat = pd.concat([a,b,c]).drop_duplicates()
    
    r = cat[(cat["Alias"] == alias) & (cat["PatientDiagnos_ModifieradDatum"] == diag_date)]
    #x = r["VårdtillfälleFörDiagnos_SlutDatum"].replace("n/a", str(distant_future), inplace=True)
    #r.loc[] = x
    return r


pats = defaultdict(list)
counts = []
counters = Counter()
for k, row in df.iterrows():
    alias = row["Alias_sk1718"]
    print(alias)
    diag_date = row["diagnosis_date"]
    r = lookup(alias, diag_date)
    diag_date = parse_iso8601_datetime(diag_date)
    c = 0
    index_date = parse_iso8601_datetime(row["pseudo"].split(" -- ")[0])
    for k, r2 in r.iterrows():
        diag = r2["PatientDiagnos_Kod"]
        if not diag.startswith(diag_prefix):
            continue
        c += 1
        vardform = r2["VårdtillfälleFörDiagnos_VardformText"]
        vt_start = parse_iso8601_datetime(r2["VårdtillfälleFörDiagnos_StartDatum"])
        vt_end = parse_iso8601_datetime(r2["VårdtillfälleFörDiagnos_SlutDatum"])
        print(f'{r2["PatientDiagnos_ModifieradDatum"]}\t{diag}\t{r2["Diagnostyp"]}\t{r2["AktivitetTyp"]}\t{vardform}')
        index_start_diff = vt_start - diag_date
        k = "O" if vardform == "Öppenvård" else "S"
        k += "_ErrEnd" if vt_end < index_date else "_OkEnd"
        #k += "_ErrStart" if index_start_diff > timedelta(days=30) else "_OkStart"
        k2 = k+"_"+r2["AktivitetTyp"]
        counters[k] += 1
        counters[k2] += 1
        pats[k2].append(alias)

    print()
    counts.append(c)
    
print(counts)
for k in sorted(counters.keys()):
    print(f'{k}\t{counters[k]}')
#print(f"Oppen: {oppen}")
#print(f'Sluten: {sluten}')
for p in pats["S_OkEnd_Akutkliniken Läk"]:
    print(p)

    

## DF check

In [ ]:
def list_cols(df):
    cols = list(df)
    for c in cols:
        nz = len(df) - df[c].isna().sum()
        unique_values = len(df[c].value_counts())
        print(f'{c}\t{df.dtypes[c]}\t--label\t{nz}\t{unique_values}')

list_cols(sos_dors)

In [ ]:
sos_dors["LK"].value_counts()

## BOTTOM